In [1]:
################ 페이지네이션 테스트 중 #################
# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (리팩토링 완료)
# - 도시 정보를 UNIFIED_CITY_INFO로 통합하여 단일 소스로 관리
#cd "/mnt/c/Users/redsk/OneDrive/デスクトップ/mikael_project/test_folder"
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')
import platform
import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용
from datetime import datetime    # 타임스탬프용
import hashlib                   # URL 재사용 방지 시스템용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "MAX_PRODUCTS_PER_CITY": 6,     #⭐⭐⭐⭐⭐⭐⭐⭐⭐#
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["다낭"]

# =============================================================================
# 📍 [최종 수정본] 단일 정보 소스 및 리팩토링된 함수
# =============================================================================

# 모든 도시의 상세 정보가 포함된 딕셔너리 (최종 확장본)
UNIFIED_CITY_INFO = {
    # 동남아시아
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "아유타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK"}, # 방콕 공항 사용
    "치앙마이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"},
    "빠이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"}, # 치앙마이 공항 사용
    "치앙라이": {"대륙": "아시아", "국가": "태국", "코드": "CEI"},
    "푸켓": {"대륙": "아시아", "국가": "태국", "코드": "HKT"},
    "피피": {"대륙": "아시아", "국가": "태국", "코드": "KBV"}, # 크라비 공항 사용
    "크라비": {"대륙": "아시아", "국가": "태국", "코드": "KBV"},
    "후아힌": {"대륙": "아시아", "국가": "태국", "코드": "HHQ"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아", "코드": "KUL"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아", "코드": "BKI"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아", "코드": "PEN"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아", "코드": "LGK"},
    "세부": {"대륙": "아시아", "국가": "필리핀", "코드": "CEB"},
    "보홀": {"대륙": "아시아", "국가": "필리핀", "코드": "TAG"},
    "마닐라": {"대륙": "아시아", "국가": "필리핀", "코드": "MNL"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀", "코드": "MPH"}, # 카티클란 공항
    "다낭": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"},
    "호이안": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"}, # 다낭 공항 사용
    "후에": {"대륙": "아시아", "국가": "베트남", "코드": "HUI"},
    "호치민": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"},
    "무이네": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"}, # 호치민 공항 사용
    "푸꾸옥": {"대륙": "아시아", "국가": "베트남", "코드": "PQC"},
    "나트랑": {"대륙": "아시아", "국가": "베트남", "코드": "CXR"},
    "하노이": {"대륙": "아시아", "국가": "베트남", "코드": "HAN"},
    "달랏": {"대륙": "아시아", "국가": "베트남", "코드": "DLI"},
    "발리": {"대륙": "아시아", "국가": "인도네시아", "코드": "DPS"},
    "프놈펜": {"대륙": "아시아", "국가": "캄보디아", "코드": "PNH"}, #아직 상품없음 25.0723
    "시엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, #아직 상품없음 25.0723
    "씨엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, # 시엠립 동의어, #아직 상품없음 25.0723
    "비엔티안": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"},
    "방비엥": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"}, # 비엔티안 공항 사용
    "루앙프라방": {"대륙": "아시아", "국가": "라오스", "코드": "LPQ"},

    # 동북아시아
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "교토": {"대륙": "아시아", "국가": "일본", "코드": "KIX"}, # 오사카 공항 사용
    "나고야": {"대륙": "아시아", "국가": "일본", "코드": "NGO"},
    "후쿠오카": {"대륙": "아시아", "국가": "일본", "코드": "FUK"},
    "벳푸": {"대륙": "아시아", "국가": "일본", "코드": "OIT"}, # 오이타 공항 사용
    "오이타": {"대륙": "아시아", "국가": "일본", "코드": "OIT"},
    "구마모토": {"대륙": "아시아", "국가": "일본", "코드": "KMJ"},
    "오키나와": {"대륙": "아시아", "국가": "일본", "코드": "OKA"},
    "미야코지마": {"대륙": "아시아", "국가": "일본", "코드": "MMY"},
    "삿포로": {"대륙": "아시아", "국가": "일본", "코드": "CTS"},
    "타이베이": {"대륙": "아시아", "국가": "대만", "코드": "TPE"},
    "상하이": {"대륙": "아시아", "국가": "중국", "코드": "PVG"},
    "베이징": {"대륙": "아시아", "국가": "중국", "코드": "PEK"},
    "하이난(싼야)": {"대륙": "아시아", "국가": "중국", "코드": "SYX"},
    "마카오": {"대륙": "아시아", "국가": "마카오", "코드": "MFM"},
    
    # 남아시아
    "뉴델리": {"대륙": "아시아", "국가": "인도", "코드": "DEL"},
    "뭄바이": {"대륙": "아시아", "국가": "인도", "코드": "BOM"},
    "카트만두": {"대륙": "아시아", "국가": "네팔", "코드": "KTM"},
    "포카라": {"대륙": "아시아", "국가": "네팔", "코드": "PKR"},

    # 한국
    "서울": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "부산": {"대륙": "아시아", "국가": "대한민국", "코드": "PUS"},
    "제주": {"대륙": "아시아", "국가": "대한민국", "코드": "CJU"},
    "대구": {"대륙": "아시아", "국가": "대한민국", "코드": "TAE"},
    "광주": {"대륙": "아시아", "국가": "대한민국", "코드": "KWJ"},
    "여수": {"대륙": "아시아", "국가": "대한민국", "코드": "RSU"},
    "인천": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "김포": {"대륙": "아시아", "국가": "대한민국", "코드": "GMP"},

    # 유럽
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "더블린": {"대륙": "유럽", "국가": "아일랜드", "코드": "DUB"},
    "로마": {"대륙": "유럽", "국가": "이탈리아", "코드": "FCO"},
    "피렌체": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR"},
    "베네치아": {"대륙": "유럽", "국가": "이탈리아", "코드": "VCE"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아", "코드": "MXP"},
    "바르셀로나": {"대륙": "유럽", "국가": "스페인", "코드": "BCN"},
    "마드리드": {"대륙": "유럽", "국가": "스페인", "코드": "MAD"},
    "세비야": {"대륙": "유럽", "국가": "스페인", "코드": "SVQ"},
    "그라나다": {"대륙": "유럽", "국가": "스페인", "코드": "GRX"},
    "이비자": {"대륙": "유럽", "국가": "스페인", "코드": "IBZ"},
    "리스본": {"대륙": "유럽", "국가": "포르투갈", "코드": "LIS"},
    "포르투": {"대륙": "유럽", "국가": "포르투갈", "코드": "OPO"},
    "프라하": {"대륙": "유럽", "국가": "체코", "코드": "PRG"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아", "코드": "VIE"},
    "취리히": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"},
    "인터라켄": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"}, # 취리히 공항 사용
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드", "코드": "AMS"},
    "브뤼셀": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU"},
    "뮌헨": {"대륙": "유럽", "국가": "독일", "코드": "MUC"},
    "베를린": {"대륙": "유럽", "국가": "독일", "코드": "BER"},
    "프랑크푸르트": {"대륙": "유럽", "국가": "독일", "코드": "FRA"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리", "코드": "BUD"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드", "코드": "WAW"},
    "크라쿠프": {"대륙": "유럽", "국가": "폴란드", "코드": "KRK"},
    "아테네": {"대륙": "유럽", "국가": "그리스", "코드": "ATH"},
    "산토리니": {"대륙": "유럽", "국가": "그리스", "코드": "JTR"},
    "자그레브": {"대륙": "유럽", "국가": "크로아티아", "코드": "ZAG"},
    "두브로브니크": {"대륙": "유럽", "국가": "크로아티아", "코드": "DBV"},
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크", "코드": "CPH"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴", "코드": "ARN"},
    "오슬로": {"대륙": "유럽", "국가": "노르웨이", "코드": "OSL"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드", "코드": "HEL"},
    "이스탄불": {"대륙": "유럽", "국가": "터키", "코드": "IST"},

    # 북미
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국", "코드": "LAX"},
    "시카고": {"대륙": "북미", "국가": "미국", "코드": "ORD"},
    "하와이": {"대륙": "북미", "국가": "미국", "코드": "HNL"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국", "코드": "SFO"},
    "라스베이거스": {"대륙": "북미", "국가": "미국", "코드": "LAS"},
    "워싱턴 D.C.": {"대륙": "북미", "국가": "미국", "코드": "IAD"},
    "보스턴": {"대륙": "북미", "국가": "미국", "코드": "BOS"},
    "시애틀": {"대륙": "북미", "국가": "미국", "코드": "SEA"},
    "밴쿠버": {"대륙": "북미", "국가": "캐나다", "코드": "YVR"},
    "토론토": {"대륙": "북미", "국가": "캐나다", "코드": "YYZ"},
    "칸쿤": {"대륙": "북미", "국가": "멕시코", "코드": "CUN"},

    # 오세아니아
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
    "멜버른": {"대륙": "오세아니아", "국가": "호주", "코드": "MEL"},
    "괌": {"대륙": "오세아니아", "국가": "괌", "코드": "GUM"},
    "사이판": {"대륙": "오세아니아", "국가": "북마리아나 제도", "코드": "SPN"},
    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "AKL"},
    
    # 중동
    "두바이": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "DXB"},
}

print(f"✅ UNIFIED_CITY_INFO 로드 완료! {len(UNIFIED_CITY_INFO)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 단일 정보 소스(UNIFIED_CITY_INFO) 사용
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (UNIFIED_CITY_INFO 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        code = info.get("코드", city_name[:3].upper())
        return code
    return city_name[:3].upper()

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (사전 정의된 값만 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 정의되지 않은 도시에 대한 기본값
        return "기타", "기타"

def get_last_product_number(city_name):
    """기존 CSV에서 마지막 번호 확인 (번호 연속성 확보)"""
    try:
        continent, country = get_city_info(city_name)
        
        # 도시별 CSV 경로
        csv_path = os.path.join("data", continent, country, city_name, f"myrealtrip_{city_name}_products.csv")
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path, encoding='utf-8-sig')
            if not df.empty and '번호' in df.columns:
                last_number = df['번호'].max()
                print(f"📊 기존 CSV 마지막 번호: {last_number}")
                return last_number
        
        print(f"📄 기존 CSV 파일 없음 - 0부터 시작")
        return -1  # 파일이 없으면 -1 반환 (다음은 0부터 시작)
        
    except Exception as e:
        print(f"⚠️ 마지막 번호 확인 실패: {e}")
        return -1

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (기존: get_product_name_by_type → 새로운: get_product_name)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """✅ 가격 수집 - 수정된 버전 (잘못된 텍스트 필터링 강화)"""
    print(f"  💰 가격 정보 수집 중...")
    
    # 마이리얼트립 실제 가격 셀렉터들
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 71, 71)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".text-red"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        # 더 구체적인 XPath
        (By.XPATH, "//span[contains(text(), '원~') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), '원-') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), ',') and contains(text(), '원') and not(contains(text(), '인원')) and not(contains(text(), '최소'))]"),
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_elements = driver.find_elements(selector_type, selector_value)
            
            for price_element in price_elements:
                found_price = price_element.text.strip()
                
                if not found_price:
                    continue
                
                # 🔥 강력한 필터링: 가격이 아닌 텍스트들 제외
                invalid_keywords = [
                    '쿠폰', '받기', '다운', '할인', '적립', '포인트',
                    '최소', '인원', '명', '최대', '선택', '옵션',
                    '예약', '신청', '문의', '상담', '확인', '명부터',
                    '시간', '일정', '코스', '투어', '여행', '부터',
                    '언어', '가이드', '포함', '불포함', '이상',
                    '취소', '환불', '변경', '안내', '모집'
                ]
                
                if any(keyword in found_price for keyword in invalid_keywords):
                    continue
                
                # 길이 제한 (너무 긴 텍스트 제외)
                if len(found_price) > 30:
                    continue
                
                # 가격 패턴 확인
                import re
                price_patterns = [
                    r'\d{1,3}(?:,\d{3})+원[~-]?',  # 10,000원~ 형태
                    r'\d+,\d+원[~-]?',             # 간단한 천단위 구분
                    r'\d{4,}원[~-]?',              # 10000원~ 형태
                ]
                
                is_valid_price = any(re.search(pattern, found_price) for pattern in price_patterns)
                
                if is_valid_price and '원' in found_price:
                    print(f"    ✅ 유효한 가격 발견: '{found_price}'")
                    return found_price
                    
        except Exception:
            continue
    
    print(f"    ❌ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

print("✅ 그룹 1 완료: 기본 설정 및 핵심 함수 정의 완료!")
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print(f"🌍 지원 도시: {len(UNIFIED_CITY_INFO)}개")

🔧 Selenium 버전: 4.25.0
✅ UNIFIED_CITY_INFO 로드 완료! 116개 도시 지원
✅ 그룹 1 완료: 기본 설정 및 핵심 함수 정의 완료!
🔢 현재 설정: 6개 상품 크롤링
🏙️ 검색 도시: ['다낭']
🌍 지원 도시: 116개


In [2]:
# =============================================================================
# 🚀 그룹 2: 이미지 처리 및 데이터 저장 함수들
# - 이미지 다운로드, 데이터 저장, 결과 처리 관련 함수들
# =============================================================================

def download_image(driver, product_name, city_name, product_number):
    """✅ 이미지 다운로드 - 계층 구조 저장 (전역 번호 사용으로 수정)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'relative_path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    # 더 광범위한 이미지 셀렉터들
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.CSS_SELECTOR, ".image-gallery img:first-child"),
        (By.CSS_SELECTOR, ".slider img:first-child"),
        (By.XPATH, "//img[@width and @height and (@width > '200' or @height > '200')]"),
        (By.XPATH, "//img[contains(@src, 'large') or contains(@src, 'big') or contains(@src, 'main')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
        (By.CSS_SELECTOR, "img[src*='cloudfront']"),
        (By.CSS_SELECTOR, "img[src*='amazonaws']"),
        (By.XPATH, "(//img[contains(@src, 'http') and not(contains(@src, 'icon')) and not(contains(@src, 'logo'))])[1]"),
        (By.XPATH, "//img[contains(@alt, '상품') or contains(@alt, '투어') or contains(@alt, '여행')]"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    if not img_url or not img_url.startswith('http'):
                        continue
                    exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile', 'button', 'arrow', 'star', 'thumb', 'small', 'mini']
                    if any(pattern in img_url.lower() for pattern in exclude_patterns):
                        continue
                    try:
                        size = img_element.size
                        if size and (size.get('width', 0) < 100 or size.get('height', 0) < 100):
                            continue
                    except:
                        pass
                    print(f"    ✅ 이미지 URL 발견: {img_url[:50]}...")
                    break
                except Exception:
                    continue
            if img_url:
                break
        except Exception:
            continue

    # ⭐️ 핵심 수정: 파일명을 product_number(전역 카운터) 기준으로 생성
    city_code = get_city_code(city_name)
    # 번호 패딩을 000 -> 0000으로 늘려 더 많은 상품 지원 (예: BCN_0001.jpg)
    img_filename = f"{city_code}_{product_number:04d}.jpg"

    if not img_url:
        print(f"    ❌ 이미지 URL을 찾을 수 없습니다")
        return {
            'status': '이미지 URL 없음',
            'filename': img_filename,
            'path': '',
            'relative_path': '',
            'size': 0
        }

    # 🏗️ 계층 구조 폴더 생성 및 이미지 다운로드
    try:
        import urllib.request
        import os
        
        # 도시 정보 가져오기
        continent, country = get_city_info(city_name)
        
        # 📁 계층 구조 폴더 경로 생성: myrealtripthumb_img/대륙/국가/도시/
        base_folder = "myrealtripthumb_img"
        hierarchical_folder = os.path.join(base_folder, continent, country, city_name)
        
        # 폴더 생성 (계층 구조)
        os.makedirs(hierarchical_folder, exist_ok=True)
        print(f"    📁 계층 폴더 생성: {hierarchical_folder}")
        
        # 전체 파일 경로
        img_path = os.path.join(hierarchical_folder, img_filename)
        
        # 상대 경로 (CSV에 저장용)
        relative_path = os.path.join(continent, country, city_name, img_filename)
        
        # User-Agent 추가로 다운로드 성공률 높이기
        req = urllib.request.Request(
            img_url,
            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        )
        
        with urllib.request.urlopen(req, timeout=10) as response:
            with open(img_path, 'wb') as f:
                f.write(response.read())
        
        file_size = os.path.getsize(img_path)
        
        if file_size < 1024:
            os.remove(img_path)
            print(f"    ❌ 이미지 파일이 너무 작습니다 ({file_size} bytes)")
            return {
                'status': '파일 너무 작음',
                'filename': img_filename,
                'path': '',
                'relative_path': '',
                'size': 0
            }
        
        print(f"    ✅ 계층 구조 이미지 저장 완료! ({file_size:,} bytes)")
        print(f"    📍 저장 위치: {relative_path}")
        
        return {
            'status': '다운로드 완료',
            'filename': img_filename,
            'path': img_path,
            'relative_path': relative_path,
            'size': file_size
        }
        
    except Exception as e:
        print(f"    ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'다운로드 실패: {type(e).__name__}',
            'filename': img_filename,
            'path': '',
            'relative_path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    df = pd.DataFrame(products_data)
    csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name
        }
    }
    
    try:
        with open('config/data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: config/data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

def safe_csv_write(file_path, df, mode='w', header=True):
    """CSV 파일을 안전하게 작성 (Permission denied 오류 해결)"""
    max_retries = 5
    
    for attempt in range(max_retries):
        try:
            # 기존 파일이 있고 쓰기 모드인 경우 백업 생성
            if mode == 'a' and os.path.exists(file_path):
                # 파일이 잠겨있는지 확인
                try:
                    with open(file_path, 'a', encoding='utf-8-sig') as test_file:
                        test_file.write('')  # 빈 문자열 쓰기 테스트
                except PermissionError:
                    print(f"    ⚠️ 파일이 잠겨있음, {attempt+1}번째 재시도...")
                    time.sleep(2)  # 2초 대기
                    continue
            
            # CSV 파일 작성
            df.to_csv(file_path, mode=mode, header=header, index=False, encoding='utf-8-sig')
            return True
            
        except PermissionError as e:
            print(f"    ⚠️ 권한 오류 ({attempt+1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                # 잠깐 대기 후 재시도
                wait_time = (attempt + 1) * 2  # 2, 4, 6, 8초
                print(f"    ⏰ {wait_time}초 대기 후 재시도...")
                time.sleep(wait_time)
            else:
                # 최종 시도 - 백업 파일로 저장
                backup_path = file_path.replace('.csv', f'_backup_{datetime.now().strftime("%H%M%S")}.csv')
                try:
                    df.to_csv(backup_path, mode='w', header=True, index=False, encoding='utf-8-sig')
                    print(f"    💾 백업 파일로 저장: {backup_path}")
                    return True
                except Exception as backup_error:
                    print(f"    ❌ 백업 저장도 실패: {backup_error}")
                    return False
                    
        except Exception as e:
            print(f"    ❌ 예상치 못한 오류: {e}")
            return False
    
    return False

def save_batch_data(batch_results, city_name):
    """배치 데이터 저장 (Permission denied 오류 해결)"""
    if not batch_results:
        return None
    
    try:
        continent, country = get_city_info(city_name)
        
        data_dir = os.path.join("data", continent, country, city_name)
        os.makedirs(data_dir, exist_ok=True)
        
        df = pd.DataFrame(batch_results)

        # 1. 도시별 CSV (안전한 저장)
        city_csv = os.path.join(data_dir, f"myrealtrip_{city_name}_products.csv")
        if os.path.exists(city_csv):
            city_success = safe_csv_write(city_csv, df, mode='a', header=False)
        else:
            city_success = safe_csv_write(city_csv, df, mode='w', header=True)

        # 2. 국가별 통합 CSV (안전한 저장)
        country_dir = os.path.join("data", continent, country)
        os.makedirs(country_dir, exist_ok=True)
        country_csv = os.path.join(country_dir, f"{country}_myrealtrip_products_all.csv")
        if os.path.exists(country_csv):
            country_success = safe_csv_write(country_csv, df, mode='a', header=False)
        else:
            country_success = safe_csv_write(country_csv, df, mode='w', header=True)

        if city_success and country_success:
            print(f"✅ 배치 데이터 저장 완료:")
            print(f"   📁 도시별: {city_csv}")
            print(f"   📁 국가별: {country_csv}")
            
            return {
                "city_csv": city_csv,
                "country_csv": country_csv,
                "data_count": len(batch_results)
            }
        else:
            print(f"⚠️ 일부 파일 저장 실패 (도시:{city_success}, 국가:{country_success})")
            return {
                "city_csv": city_csv if city_success else "저장실패",
                "country_csv": country_csv if country_success else "저장실패",
                "data_count": len(batch_results)
            }
        
    except Exception as e:
        print(f"❌ 배치 데이터 저장 실패: {e}")
        return None

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집 - 수정된 버전 (후기 내용 제외)"""
    print(f"  🌐 언어 정보 찾는 중...")
    
    # 언어 정보가 있을 가능성이 높은 셀렉터들
    language_selectors = [
        # 언어 섹션 직접 타겟팅
        (By.XPATH, "//dt[contains(text(), '언어')]/following-sibling::dd"),
        (By.XPATH, "//span[contains(@class, 'language')]"),
        (By.XPATH, "//div[contains(@class, 'language')]//span"),
        # 짧은 언어 텍스트만 (후기 제외)
        (By.XPATH, "//span[contains(text(), '한국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '영어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '중국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '일본어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'Korean') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'English') and string-length(text()) < 20]"),
        # 언어 아이콘 근처의 텍스트
        (By.XPATH, "//i[contains(@class, 'language')]/following-sibling::span"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_elements = driver.find_elements(selector_type, selector_value)
            
            for language_element in language_elements:
                language_text = language_element.text.strip()
                
                if not language_text:
                    continue
                
                # 🔥 강력한 필터링: 후기나 긴 텍스트 제외
                # 길이 제한 (언어 정보는 보통 짧음)
                if len(language_text) > 50:
                    continue
                
                # 후기 관련 키워드 제외
                review_keywords = [
                    '후기', '리뷰', '평가', '별점', '추천', '만족',
                    '여행', '투어', '경험', '좋았', '나빴', '최고',
                    '다음에', '또', '재방문', '친절', '서비스',
                    '가격', '시간', '일정', '코스', '가이드',
                    '예약', '신청', '문의', '확인', '취소'
                ]
                
                if any(keyword in language_text for keyword in review_keywords):
                    continue
                
                # 언어 키워드 확인
                language_keywords = [
                    '언어', '한국어', '영어', '중국어', '일본어', 
                    'Korean', 'English', 'Chinese', 'Japanese',
                    '가능', '지원', '제공'
                ]
                
                if any(keyword in language_text for keyword in language_keywords):
                    print(f"    ✅ 언어 정보 발견: {language_text}")
                    return language_text
                    
        except Exception:
            continue

    print(f"    ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

print("✅ 그룹 2 완료: 이미지 처리 및 데이터 저장 함수들 정의 완료!")
print("🔧 제공된 기능:")
print("   - download_image(): 계층 구조 이미지 다운로드")
print("   - save_results(): 데이터 저장")
print("   - safe_csv_write(): 안전한 CSV 저장")
print("   - save_batch_data(): 배치 데이터 저장")
print("   - get_rating(): 평점 수집")
print("   - get_review_count(): 리뷰 수 수집")
print("   - get_language(): 언어 정보 수집")

✅ 그룹 2 완료: 이미지 처리 및 데이터 저장 함수들 정의 완료!
🔧 제공된 기능:
   - download_image(): 계층 구조 이미지 다운로드
   - save_results(): 데이터 저장
   - safe_csv_write(): 안전한 CSV 저장
   - save_batch_data(): 배치 데이터 저장
   - get_rating(): 평점 수집
   - get_review_count(): 리뷰 수 수집
   - get_language(): 언어 정보 수집


In [3]:

# 🔄 그룹 3: 상태 관리 시스템 구축 (데이터 연속성 확보) - url_history 구조 적용
# - URL 재사용 방지, 세션 안전성, 크롤링 상태 관리, url history 폴더와 파일 생성
# =============================================================================

def ensure_config_directory():
    """config 디렉토리 안정성 확보"""
    import os
    
    # 현재 작업 디렉토리의 절대 경로 확보
    current_dir = os.path.abspath(os.getcwd())
    config_dir = os.path.join(current_dir, "config")
    
    # config 디렉토리 생성 시도
    try:
        os.makedirs(config_dir, exist_ok=True)
        
        # 쓰기 권한 테스트
        test_file = os.path.join(config_dir, "write_test.tmp")
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        
        return config_dir
        
    except (OSError, PermissionError) as e:
        # 대체 경로 시도 (임시 디렉토리)
        import tempfile
        fallback_dir = os.path.join(tempfile.gettempdir(), "myrealtrip_config")
        os.makedirs(fallback_dir, exist_ok=True)
        print(f"⚠️ 기본 config 디렉토리 접근 실패, 대체 경로 사용: {fallback_dir}")
        return fallback_dir

def validate_completed_urls(config_dir):
    """completed_urls.log 파일 유효성 검증 및 정리"""
    urls_file = os.path.join(config_dir, "completed_urls.log")
    
    if not os.path.exists(urls_file):
        return set()
    
    try:
        with open(urls_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # URL 정리 및 중복 제거
        valid_urls = set()
        for line in lines:
            url = line.strip()
            if url and url.startswith('http') and '/products/' in url:
                valid_urls.add(url)
        
        # 정리된 URL로 파일 재작성
        with open(urls_file, 'w', encoding='utf-8') as f:
            for url in sorted(valid_urls):
                f.write(url + '\n')
        
        print(f"✅ completed_urls.log 정리 완료: {len(valid_urls)}개 고유 URL")
        return valid_urls
        
    except Exception as e:
        print(f"⚠️ URL 파일 검증 실패: {e}")
        return set()

def get_session_url_fingerprint(urls):
    """URL 목록으로 세션 고유 지문 생성"""
    if not urls:
        return None
    
    # URL들을 정렬하여 일관된 해시 생성
    sorted_urls = sorted(set(urls))
    urls_string = '|'.join(sorted_urls)
    
    return hashlib.md5(urls_string.encode('utf-8')).hexdigest()[:16]

def check_session_overlap(city_name, current_urls, config_dir=None):
    """이전 세션과의 URL 중복 확인 (url_history 구조 적용)"""
    # ✅ 새로운 구조: url_history 폴더 사용
    os.makedirs("url_history", exist_ok=True)
    session_history_file = os.path.join("url_history", f"{city_name}.json")
    
    if not os.path.exists(session_history_file):
        return False, "첫 번째 세션"
    
    try:
        with open(session_history_file, 'r', encoding='utf-8') as f:
            history = json.load(f)
        
        current_fingerprint = get_session_url_fingerprint(current_urls)
        
        for session in history.get('sessions', []):
            if session.get('url_fingerprint') == current_fingerprint:
                return True, f"세션 {session.get('session_id', 'unknown')}와 중복"
        
        return False, "새로운 세션"
        
    except Exception as e:
        print(f"⚠️ 세션 히스토리 확인 실패: {e}")
        return False, "확인 불가"

def verify_url_recorded(url, config_dir):
    """특정 URL이 completed_urls.log에 기록되었는지 확인"""
    urls_file = os.path.join(config_dir, "completed_urls.log")
    
    if not os.path.exists(urls_file):
        return False
    
    try:
        with open(urls_file, 'r', encoding='utf-8') as f:
            recorded_urls = set(line.strip() for line in f if line.strip())
        return url in recorded_urls
    except Exception:
        return False

def safe_url_record(url, config_dir):
    """URL을 안전하게 completed_urls.log에 기록"""
    urls_file = os.path.join(config_dir, "completed_urls.log")
    
    # 중복 확인
    if verify_url_recorded(url, config_dir):
        return True  # 이미 기록됨
    
    try:
        with open(urls_file, 'a', encoding='utf-8') as f:
            f.write(url + '\n')
        return True
    except Exception as e:
        print(f"❌ URL 기록 실패: {e}")
        return False

def collect_urls_with_session_safety(driver, city_name, completed_urls):
    """세션 안전성을 보장하는 URL 수집 (url_history 구조 적용)"""
    config_dir = ensure_config_directory()
    
    # 1. 기본 URL 수집
    try:
        from selenium.webdriver.common.by import By
        import time
        
        time.sleep(3)
        product_elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
        
        all_urls = []
        for element in product_elements:
            try:
                url = element.get_attribute('href')
                if url and ('products/' in url or 'offers/' in url) and url.startswith('http'):
                    all_urls.append(url)
            except:
                continue
        
        # 중복 제거
        all_urls = list(set(all_urls))
        
    except Exception as e:
        print(f"❌ 기본 URL 수집 실패: {e}")
        return []
    
    # 2. 세션 중복 검사
    is_duplicate, reason = check_session_overlap(city_name, all_urls, config_dir)
    if is_duplicate:
        print(f"⚠️ 세션 중복 감지: {reason}")
        return []
    
    # 3. 완료된 URL 필터링
    new_urls = [url for url in all_urls if url not in completed_urls]
    
    # 4. 세션 히스토리 업데이트 (url_history 구조 적용)
    try:
        # ✅ 새로운 구조: url_history 폴더 사용
        os.makedirs("url_history", exist_ok=True)
        session_history_file = os.path.join("url_history", f"{city_name}.json")
        
        history_data = {"sessions": []}
        if os.path.exists(session_history_file):
            with open(session_history_file, 'r', encoding='utf-8') as f:
                history_data = json.load(f)
        
        new_session = {
            "session_id": datetime.now().strftime('%Y%m%d_%H%M%S'),
            "timestamp": datetime.now().isoformat(),
            "city": city_name,
            "continent": get_city_info(city_name)[0],
            "country": get_city_info(city_name)[1],
            "url_count": len(all_urls),
            "new_url_count": len(new_urls),
            "url_fingerprint": get_session_url_fingerprint(all_urls),
            "sample_urls": all_urls[:5] if all_urls else []
        }
        
        history_data["sessions"].append(new_session)
        
        with open(session_history_file, 'w', encoding='utf-8') as f:
            json.dump(history_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ URL 히스토리 업데이트: url_history/{city_name}.json")
            
    except Exception as e:
        print(f"⚠️ 세션 히스토리 저장 실패: {e}")
    
    return new_urls

def load_crawler_state():
    """크롤링 상태 로드 (절대 경로 기반)"""
    config_dir = ensure_config_directory()
    state_file = os.path.join(config_dir, "crawler_meta.json")
    
    # 기본 상태
    default_state = {
        "total_collected_count": 0,
        "last_crawled_page": 1,
        "current_session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # 상태 파일 로드
    if os.path.exists(state_file):
        try:
            with open(state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            print(f"✅ 상태 파일 로드: {state['total_collected_count']}개 수집 완료")
        except Exception as e:
            print(f"⚠️ 상태 파일 로드 실패: {e}, 기본값 사용")
            state = default_state
    else:
        state = default_state
        print("🆕 새로운 크롤링 세션 시작")
    
    # 완료된 URL 목록 검증 및 로드
    completed_urls = validate_completed_urls(config_dir)
    print(f"✅ 완료된 URL {len(completed_urls)}개 로드")
    
    return state, completed_urls

def save_crawler_state(state, new_url=None):
    """크롤링 상태 저장 (절대 경로 기반)"""
    config_dir = ensure_config_directory()
    state_file = os.path.join(config_dir, "crawler_meta.json")
    
    # 상태 업데이트
    state["last_updated"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        # 상태 파일 저장
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(state, f, ensure_ascii=False, indent=2)
        
        # 새 URL이 있으면 안전하게 기록
        if new_url:
            safe_url_record(new_url, config_dir)
        
        return True
    except Exception as e:
        print(f"❌ 상태 저장 실패: {e}")
        return False

def filter_new_urls(all_urls, completed_urls):
    """완료되지 않은 새로운 URL만 필터링"""
    new_urls = [url for url in all_urls if url not in completed_urls]
    
    print(f"🔍 URL 필터링 결과:")
    print(f"   📊 전체 URL: {len(all_urls)}개")
    print(f"   ✅ 완료된 URL: {len(completed_urls)}개")
    print(f"   🆕 새로운 URL: {len(new_urls)}개")
    
    return new_urls

def collect_all_24_urls(driver):
    """24개 URL 수집 (Products + Offers)"""
    print("📊 현재 페이지의 모든 URL 수집 중...")
    
    # 더 포괄적인 셀렉터로 모든 링크 수집
    all_selectors = [
        "a[href*='/products/']",  # Products
        "a[href*='/offers/']",    # Offers
    ]
    
    collected_urls = []
    
    for selector in all_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    url = element.get_attribute('href')
                    if url and url.startswith('http') and url not in collected_urls:
                        # Products 또는 Offers 타입 구분
                        if '/products/' in url or '/offers/' in url:
                            collected_urls.append(url)
                except:
                    continue
        except:
            continue
    
    # 중복 제거
    unique_urls = list(set(collected_urls))
    
    print(f"✅ 총 {len(unique_urls)}개 URL 수집 완료!")
    
    # URL 타입별 통계
    products = [url for url in unique_urls if '/products/' in url]
    offers = [url for url in unique_urls if '/offers/' in url]
    
    print(f"   🛍️ Products: {len(products)}개")
    print(f"   🏷️ Offers: {len(offers)}개")
    
    return unique_urls

print("✅ 그룹 3 완료: 상태 관리 시스템 구축 완료! (url_history 구조 적용)")
print("🔧 제공된 기능:")
print("   - ensure_config_directory(): 안전한 디렉토리 관리")
print("   - validate_completed_urls(): URL 파일 검증 및 정리")
print("   - collect_urls_with_session_safety(): 세션 안전 URL 수집")
print("   - check_session_overlap(): 세션 중복 방지")
print("   - load_crawler_state(): 크롤링 상태 로드")
print("   - save_crawler_state(): 크롤링 상태 저장")
print("   - filter_new_urls(): 새로운 URL 필터링")
print("   - collect_all_24_urls(): 기본 URL 수집")
print("   - show_url_history_summary(): URL 히스토리 현황")
print("🛡️ URL 재사용 방지 시스템 및 세션 안전성 확보 완료!")
print("📁 새로운 구조: url_history/{도시명}.json 형태로 관리")

✅ 그룹 3 완료: 상태 관리 시스템 구축 완료! (url_history 구조 적용)
🔧 제공된 기능:
   - ensure_config_directory(): 안전한 디렉토리 관리
   - validate_completed_urls(): URL 파일 검증 및 정리
   - collect_urls_with_session_safety(): 세션 안전 URL 수집
   - check_session_overlap(): 세션 중복 방지
   - load_crawler_state(): 크롤링 상태 로드
   - save_crawler_state(): 크롤링 상태 저장
   - filter_new_urls(): 새로운 URL 필터링
   - collect_all_24_urls(): 기본 URL 수집
   - show_url_history_summary(): URL 히스토리 현황
🛡️ URL 재사용 방지 시스템 및 세션 안전성 확보 완료!
📁 새로운 구조: url_history/{도시명}.json 형태로 관리


In [4]:
# =============================================================================
# 🚀 그룹 4: 확장성 개선 시스템 (리팩토링된 버전)
# - 도시 관리, 페이지네이션 분석, 시스템 초기화
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장 (UNIFIED_CITY_INFO 기반)"""
    enhanced_city_data = {
        "version": "3.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(UNIFIED_CITY_INFO)
    }

    for city_name, info in UNIFIED_CITY_INFO.items():
        enhanced_city_data["cities"][city_name] = {
            "code": info.get("코드", "N/A"),
            "continent": info.get("대륙", "기타"),
            "country": info.get("국가", "기타")
        }
    
    try:
        with open('config/city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ config/city_codes.json 파일 생성 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드 (UNIFIED_CITY_INFO와 동기화)"""
    if not os.path.exists('config/city_codes.json'):
        print("📝 config/city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
    
    try:
        with open('config/city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_cities = city_data.get("cities", {})
        
        for city, info in loaded_cities.items():
            if city not in UNIFIED_CITY_INFO:
                 UNIFIED_CITY_INFO[city] = {
                     "대륙": info.get("continent"),
                     "국가": info.get("country"),
                     "코드": info.get("code")
                 }
        
        print(f"✅ config/city_codes.json 로드 및 동기화 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 코드의 UNIFIED_CITY_INFO를 사용합니다.")

def add_new_city(city_name, airport_code, continent="기타", country="기타", update_file=True):
    """새로운 도시를 UNIFIED_CITY_INFO에 추가하는 함수"""
    global UNIFIED_CITY_INFO
    
    if city_name not in UNIFIED_CITY_INFO:
        UNIFIED_CITY_INFO[city_name] = {
            "대륙": continent,
            "국가": country,
            "코드": airport_code
        }
        print(f"✅ 메모리에 추가: {city_name} → {airport_code} ({continent}, {country})")
        if update_file:
            create_city_codes_file()
        return True
    else:
        print(f"이미 존재하는 도시입니다: {city_name}")
        return False

def show_supported_cities():
    """지원하는 도시 목록 표시 (UNIFIED_CITY_INFO 기반)"""
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    cities_by_continent = {}
    for city, info in UNIFIED_CITY_INFO.items():
        continent = info.get("대륙", "기타")
        if continent not in cities_by_continent:
            cities_by_continent[continent] = []
        cities_by_continent[continent].append(city)

    for continent, cities in sorted(cities_by_continent.items()):
        print(f"\n📍 {continent}:")
        for city in sorted(cities):
            code = UNIFIED_CITY_INFO[city].get("코드", "N/A")
            print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(UNIFIED_CITY_INFO)}개 도시 지원")
    print("="*50)

def validate_city(city_name):
    """도시명 유효성 검사 (UNIFIED_CITY_INFO 기반)"""
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in UNIFIED_CITY_INFO:
        code = UNIFIED_CITY_INFO[city_name].get("코드", "N/A")
        return True, f"지원하는 도시입니다. ({code})"
    
    similar_cities = [c for c in UNIFIED_CITY_INFO if city_name.lower() in c.lower() or c.lower() in city_name.lower()]
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    global CONFIG
    
    scalability_config = {
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def analyze_pagination(driver):
    """페이지네이션 정보 분석 - 총 페이지 수, 상품 수 파악"""
    print(f"  🔍 페이지네이션 정보 분석 중...")
    
    try:
        # 페이지 로딩 완료 대기
        time.sleep(3)
        
        # 총 상품 수 찾기
        total_products = 0
        total_selectors = [
            "//span[contains(text(), '총') and contains(text(), '개')]",
            "//span[contains(text(), '전체') and contains(text(), '개')]", 
            "//div[contains(@class, 'total') or contains(@class, 'count')]//span",
            "//span[contains(text(), '결과')]",
        ]
        
        for selector in total_selectors:
            try:
                elements = driver.find_elements(By.XPATH, selector)
                for element in elements:
                    text = element.text.strip()
                    if '개' in text and any(char.isdigit() for char in text):
                        # 숫자 추출
                        import re
                        numbers = re.findall(r'\d+', text)
                        if numbers:
                            total_products = int(numbers[0])
                            print(f"    ✅ 총 상품 수 발견: {total_products}개")
                            break
                if total_products > 0:
                    break
            except:
                continue
        
        # 페이지네이션 정보 찾기
        total_pages = 1
        has_next_button = False
        
        # 다음 페이지 버튼 찾기
        next_button_selectors = [
            "//button[contains(@aria-label, '다음')]",
            "//button[contains(text(), '다음')]",
            "//a[contains(@aria-label, '다음')]", 
            "//a[contains(text(), '다음')]",
            "//button[contains(@class, 'next')]",
            "//a[contains(@class, 'next')]",
            ".pagination .next",
            ".pager .next"
        ]
        
        for selector in next_button_selectors:
            try:
                if selector.startswith('//'):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    
                for element in elements:
                    if element.is_enabled() and element.is_displayed():
                        has_next_button = True
                        print(f"    ✅ '다음 페이지' 버튼 발견!")
                        break
                if has_next_button:
                    break
            except:
                continue
        
        # 페이지 번호 찾기 (총 페이지 수 추정)
        page_number_selectors = [
            "//button[contains(@class, 'page') or contains(@class, 'pagination')]//span",
            "//a[contains(@class, 'page') or contains(@class, 'pagination')]//span",
            ".pagination button span",
            ".pager a span"
        ]
        
        max_page = 1
        for selector in page_number_selectors:
            try:
                if selector.startswith('//'):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    
                for element in elements:
                    text = element.text.strip()
                    if text.isdigit():
                        page_num = int(text)
                        max_page = max(max_page, page_num)
            except:
                continue
                
        total_pages = max_page
        
        # 페이지당 상품 수 추정 (현재 페이지 기준)
        products_per_page = 24  # 기본값
        if total_products > 0 and total_pages > 0:
            products_per_page = min(24, total_products // total_pages + (1 if total_products % total_pages > 0 else 0))
        
        return {
            'total_products': total_products,
            'total_pages': total_pages, 
            'products_per_page': products_per_page,
            'has_next_button': has_next_button,
            'is_pagination_available': has_next_button or total_pages > 1
        }
        
    except Exception as e:
        print(f"    ❌ 페이지네이션 분석 실패: {e}")
        return {
            'total_products': 0,
            'total_pages': 1,
            'products_per_page': 24,
            'has_next_button': False,
            'is_pagination_available': False
        }

def check_next_button(driver):
    """다음 페이지 버튼 작동 확인"""
    print(f"  🔍 다음 페이지 버튼 작동성 확인 중...")
    
    next_button_selectors = [
        "//button[contains(@aria-label, '다음') and not(@disabled)]",
        "//button[contains(text(), '다음') and not(@disabled)]",
        "//a[contains(@aria-label, '다음')]",
        "//a[contains(text(), '다음')]",
        "//button[contains(@class, 'next') and not(@disabled)]",
        "//a[contains(@class, 'next')]"
    ]
    
    for selector in next_button_selectors:
        try:
            element = driver.find_element(By.XPATH, selector)
            if element.is_enabled() and element.is_displayed():
                # 클릭 가능한지 확인 (실제로 클릭하지는 않음)
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", element)
                    print(f"    ✅ 다음 페이지 버튼이 작동 가능합니다!")
                    return True
                except:
                    continue
        except:
            continue
    
    print(f"    ❌ 작동 가능한 다음 페이지 버튼을 찾을 수 없습니다.")
    return False

def generate_crawling_plan(pagination_info, city_name):
    """크롤링 계획 생성 및 보고"""
    print(f"\n📋 크롤링 계획 수립 중...")
    
    plan = {
        'city': city_name,
        'total_products': pagination_info['total_products'],
        'total_pages': pagination_info['total_pages'],
        'products_per_page': pagination_info['products_per_page'],
        'pagination_available': pagination_info['is_pagination_available'],
        'estimated_time_minutes': 0,
        'recommended_batch_size': CONFIG['MAX_PRODUCTS_PER_CITY'],
        'strategy': '단일 페이지'
    }
    
    # 예상 소요 시간 계산 (상품당 약 30초 추정)
    products_to_crawl = min(pagination_info['total_products'], CONFIG['MAX_PRODUCTS_PER_CITY'])
    plan['estimated_time_minutes'] = products_to_crawl * 0.5  # 상품당 30초
    
    # 전략 결정
    if pagination_info['is_pagination_available'] and pagination_info['total_pages'] > 1:
        plan['strategy'] = '다중 페이지 순회'
        if pagination_info['total_products'] > CONFIG['MAX_PRODUCTS_PER_CITY']:
            plan['strategy'] += f" (최대 {CONFIG['MAX_PRODUCTS_PER_CITY']}개 제한)"
    
    return plan

def report_reconnaissance_results(plan):
    """정찰 결과 보고"""
    print(f"\n🔍 === 정찰 완료 보고서 ===")
    print(f"📍 도시: {plan['city']}")
    print(f"📊 발견된 총 상품 수: {plan['total_products']}개")
    print(f"📄 총 페이지 수: {plan['total_pages']}페이지")
    print(f"📋 페이지당 상품 수: {plan['products_per_page']}개")
    print(f"🔄 페이지네이션 가능: {'✅ 예' if plan['pagination_available'] else '❌ 아니오'}")
    print(f"⏱️ 예상 소요 시간: {plan['estimated_time_minutes']:.1f}분")
    print(f"🎯 크롤링 전략: {plan['strategy']}")
    print(f"📦 실제 수집 예정: {min(plan['total_products'], plan['recommended_batch_size'])}개")
    print(f"=" * 50)
    
    if plan['pagination_available']:
        print(f"🚀 페이지네이션을 활용한 전체 크롤링이 가능합니다!")
        return True
    else:
        print(f"⚠️ 페이지네이션이 제한적입니다. 현재 페이지만 크롤링 가능합니다.")
        return False

def collect_page_urls_enhanced(driver):
    """개선된 URL 수집 + 페이지네이션 정보 포함"""
    print(f"  🔍 개선된 URL 수집 시작...")
    
    # 기존 URL 수집 로직
    collected_urls = collect_all_24_urls(driver)
    
    # 페이지네이션 정보 분석
    pagination_info = analyze_pagination(driver)
    
    return collected_urls, pagination_info

def initialize_file_system():
    """파일 시스템 초기화 및 설정 (리팩토링된 버전)"""
    print("🔧 그룹 4: 확장성 개선 시스템 초기화...")
    
    update_config_for_scalability()
    
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        load_city_codes_from_file()
    
    print("✅ 그룹 4 시스템 초기화 완료!")
    return True

# 자동 초기화 실행
try:
    initialize_file_system()
    print("✅ 그룹 4 완료: 확장성 개선 시스템 구축 완료!")
    print("🔧 제공된 기능:")
    print("   - create_city_codes_file(): 도시 코드 JSON 파일 생성")
    print("   - add_new_city(): 새 도시 추가")
    print("   - show_supported_cities(): 지원 도시 목록 표시")
    print("   - validate_city(): 도시명 유효성 검사")
    print("   - analyze_pagination(): 페이지네이션 정보 분석")
    print("   - check_next_button(): 다음 페이지 버튼 확인")
    print("   - generate_crawling_plan(): 크롤링 계획 수립")
    print("   - report_reconnaissance_results(): 정찰 결과 보고")
    print("🎯 페이지네이션 자동화를 위한 준비 완료!")
    
except Exception as e:
    print(f"❌ 그룹 4 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

🔧 그룹 4: 확장성 개선 시스템 초기화...
⚙️ CONFIG 확장성 설정 업데이트 완료!
✅ config/city_codes.json 로드 및 동기화 완료! (116개 도시)
📅 마지막 업데이트: 2025-07-23 19:08:54
✅ 그룹 4 시스템 초기화 완료!
✅ 그룹 4 완료: 확장성 개선 시스템 구축 완료!
🔧 제공된 기능:
   - create_city_codes_file(): 도시 코드 JSON 파일 생성
   - add_new_city(): 새 도시 추가
   - show_supported_cities(): 지원 도시 목록 표시
   - validate_city(): 도시명 유효성 검사
   - analyze_pagination(): 페이지네이션 정보 분석
   - check_next_button(): 다음 페이지 버튼 확인
   - generate_crawling_plan(): 크롤링 계획 수립
   - report_reconnaissance_results(): 정찰 결과 보고
🎯 페이지네이션 자동화를 위한 준비 완료!


In [5]:
# =============================================================================
# 🛠️ 그룹 5: 브라우저 제어 및 유틸리티 함수들
# - 드라이버 설정, 페이지 네비게이션, 유틸리티 기능들
# =============================================================================

def make_user_agent(ua, is_mobile):
    """User Agent 생성 함수"""
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    """랜덤 지리적 위치 생성"""
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        # shutil.rmtree(raw_path) # 폴더 삭제 방지를 위해 이 줄을 주석 처리
        pass
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
        print(platform.system())
    except Exception as e:
        print('\n',"-"*50,'\n',"-"*50,'\n')
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,'\n',"-"*50,'\n')
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].click();", view_all_button)
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print(f"  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def safe_browser_restart():
    """안전한 브라우저 재시작 with 3번 재시도"""
    global driver
    
    for attempt in range(3):  # 3번 시도
        try:
            print(f"🔄 브라우저 재시작 시도 {attempt+1}/3...")
            
            # 1단계: 안전한 종료
            if 'driver' in globals() and driver:
                driver.quit()
                driver = None
            
            # 2단계: 대기 및 정리
            wait_time = random.uniform(5, 10)
            print(f"⏰ {wait_time:.1f}초 대기 중...")
            time.sleep(wait_time)
            
            # 3단계: 새 브라우저 시작
            print("🚀 새 브라우저 시작 중...")
            driver = setup_driver()
            
            # 4단계: 동작 검증
            print("🔍 브라우저 동작 검증 중...")
            driver.get("https://www.myrealtrip.com/")
            time.sleep(2)
            
            print("✅ 브라우저 재시작 성공!")
            return True, "재시작 성공"
            
        except Exception as e:
            print(f"❌ 재시작 시도 {attempt+1} 실패: {type(e).__name__}: {e}")
            if attempt == 2:  # 마지막 시도
                print("🚨 브라우저 재시작 최종 실패!")
                return False, f"재시작 불가: {e}"
            print(f"🔄 {3-attempt-1}초 후 재시도...")
            time.sleep(3)  # 다음 시도 전 대기
    
    return False, "최종 실패"

def return_to_current_page():
    """현재 설정된 도시 상품 목록 페이지로 복귀 (수정됨)"""
    try:
        current_city = CITIES_TO_SEARCH[0] if CITIES_TO_SEARCH else "바르셀로나"
        print(f"🔄 {current_city} 상품 목록 페이지로 복귀 중...")
        
        # 동적 URL 생성 (바르셀로나 기준)
        if current_city == "바르셀로나":
            search_url = "https://www.myrealtrip.com/experiences/?destination=%EB%B0%94%EB%A5%B4%EC%85%80%EB%A1%9C%EB%82%98"
        else:
            # 다른 도시의 경우 검색 페이지로 이동
            search_url = f"https://www.myrealtrip.com/experiences/?destination={current_city}"
        
        driver.get(search_url)
        time.sleep(random.uniform(3, 5))
        print("✅ 페이지 복귀 완료")
        return True
    except Exception as e:
        print(f"❌ 페이지 복귀 실패: {e}")
        return False

# =============================================================================
# 🛠️ 유틸리티 함수들 (진행률 표시, 재시도 로직 등)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG.get("SAVE_INTERMEDIATE", False):
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

print("✅ 그룹 5 완료: 브라우저 제어 및 유틸리티 함수들 정의 완료!")
print("🔧 제공된 기능:")
print("   - setup_driver(): 크롬 드라이버 설정")
print("   - go_to_main_page(): 메인 페이지 이동")
print("   - find_and_fill_search(): 검색창 입력")
print("   - click_search_button(): 검색 버튼 클릭")
print("   - handle_popup(): 팝업 처리")
print("   - click_view_all(): 전체 상품 보기")
print("   - collect_page_urls(): 상품 URL 수집")
print("   - safe_browser_restart(): 안전한 브라우저 재시작")
print("   - return_to_current_page(): 페이지 복귀")
print("   - print_progress(): 진행률 표시")
print("   - retry_operation(): 재시도 로직")
print("   - make_safe_filename(): 안전한 파일명 생성")
print("🛡️ 브라우저 안정성 및 사용자 경험 개선 완료!")

✅ 그룹 5 완료: 브라우저 제어 및 유틸리티 함수들 정의 완료!
🔧 제공된 기능:
   - setup_driver(): 크롬 드라이버 설정
   - go_to_main_page(): 메인 페이지 이동
   - find_and_fill_search(): 검색창 입력
   - click_search_button(): 검색 버튼 클릭
   - handle_popup(): 팝업 처리
   - click_view_all(): 전체 상품 보기
   - collect_page_urls(): 상품 URL 수집
   - safe_browser_restart(): 안전한 브라우저 재시작
   - return_to_current_page(): 페이지 복귀
   - print_progress(): 진행률 표시
   - retry_operation(): 재시도 로직
   - make_safe_filename(): 안전한 파일명 생성
🛡️ 브라우저 안정성 및 사용자 경험 개선 완료!


In [6]:
# =============================================================================
# 🚀 그룹 6: 드라이버 초기화 및 기본 설정
# - 드라이버 시작, 이미지 폴더 설정, 기본 환경 구축
# =============================================================================

print("🚀 마이리얼트립 크롤링 시스템 시작!")
print("=" * 80)

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 드라이버 초기화
try:
    # 기존 드라이버가 있다면 상태 확인
    try:
        current_url = driver.current_url
        print("✅ 기존 드라이버 감지됨 - 재사용 가능한지 확인 중...")
        
        # 간단한 테스트로 드라이버 작동 확인
        driver.execute_script("return document.readyState;")
        print("✅ 기존 드라이버 정상 작동 중! 재사용합니다.")
        
    except (NameError, WebDriverException):
        print("🆕 새로운 드라이버 초기화 중...")
        driver = setup_driver()
        print("✅ 드라이버 초기화 완료!")
        
except Exception as e:
    print(f"❌ 드라이버 초기화 실패: {e}")
    print("🔄 드라이버 재생성 시도...")
    try:
        driver = setup_driver()
        print("✅ 드라이버 재생성 성공!")
    except Exception as retry_error:
        print(f"❌ 드라이버 재생성도 실패: {retry_error}")
        raise

# ✅ 수정: 이미지 폴더 연속성 확보 - 기존 이미지 보존
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.join(os.path.abspath(""), "myrealtripthumb_img")
    # ✅ 핵심 수정: 이미지 폴더 삭제 코드 완전 제거 (데이터 연속성 확보)
    # shutil.rmtree(img_folder_path)  # 이 줄을 제거하여 기존 이미지 보존
    os.makedirs(img_folder_path, exist_ok=True)
    print(f"📁 이미지 폴더 확인 완료 (기존 이미지 연속성 보장): {img_folder_path}")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("=" * 60)
print(f"🌏 설정된 검색 도시: {city_name}")
print(f"  🌍 대륙: {continent}")
print(f"  🏛️ 국가: {country}")  
print(f"  ✈️ 공항 코드: {get_city_code(city_name)}")
print(f"⚙️ 크롤링 설정:")
print(f"  📊 최대 상품 수: {CONFIG['MAX_PRODUCTS_PER_CITY']}개")
print(f"  ⏱️ 재시도 횟수: {CONFIG['RETRY_COUNT']}회")
print(f"  🔄 대기 시간: {CONFIG['MIN_DELAY']}-{CONFIG['MAX_DELAY']}초")
print(f"  🖼️ 이미지 저장: {'✅ 활성화' if CONFIG['SAVE_IMAGES'] else '❌ 비활성화'}")
print("=" * 60)

# 도시 유효성 검증
is_valid, message = validate_city(city_name)
if is_valid:
    print(f"✅ 도시 유효성 검증: {message}")
else:
    print(f"⚠️ 도시 유효성 경고: {message}")
    print("💡 계속 진행하지만 예상치 못한 결과가 발생할 수 있습니다.")

# 데이터 저장 경로 미리 생성
try:
    data_dir = os.path.join("data", continent, country, city_name)
    os.makedirs(data_dir, exist_ok=True)
    print(f"📁 데이터 저장 경로 생성 완료: {data_dir}")
    
    # 국가별 통합 폴더도 생성
    country_dir = os.path.join("data", continent, country)
    os.makedirs(country_dir, exist_ok=True)
    print(f"📁 국가별 통합 경로 생성 완료: {country_dir}")
    
except Exception as e:
    print(f"⚠️ 데이터 폴더 생성 실패: {e}")
    print("💡 크롤링은 계속 진행되지만 저장 시 문제가 발생할 수 있습니다.")

# 상태 관리 시스템 초기화
print(f"\n🔄 상태 관리 시스템 초기화 중...")
try:
    crawler_state, completed_urls = load_crawler_state()
    print(f"✅ 상태 관리 시스템 로드 완료")
    print(f"  📊 이전 수집 완료: {crawler_state.get('total_collected_count', 0)}개")
    print(f"  📝 완료된 URL: {len(completed_urls)}개")
    
    # 🆕 번호 연속성 확보: 기존 CSV 마지막 번호 확인
    last_product_number = get_last_product_number(city_name)
    start_number = last_product_number + 1  # 다음 번호부터 시작
    
    print(f"🔢 번호 연속성 설정:")
    print(f"  📊 기존 마지막 번호: {last_product_number}")
    print(f"  🆕 시작 번호: {start_number}")
    
except Exception as e:
    print(f"⚠️ 상태 관리 시스템 초기화 실패: {e}")
    print("💡 기본 상태로 진행합니다.")
    crawler_state = {
        "total_collected_count": 0,
        "last_crawled_page": 1,
        "current_session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    completed_urls = set()
    start_number = 0

print(f"\n🎉 그룹 6 완료: 시스템 초기화 성공!")
print(f"📍 위치: {continent} > {country} > {city_name}")
print(f"📁 이미지 저장 기본 경로: myrealtripthumb_img/{continent}/{country}/{city_name}/")
print(f"📁 데이터 저장 경로: data/{continent}/{country}/{city_name}/")
print(f"🔢 번호 시작점: {start_number}")
print("🚀 다음: 그룹 7을 실행하여 웹사이트 검색을 시작하세요!")
print("=" * 80)

🚀 마이리얼트립 크롤링 시스템 시작!
🔄 결과 저장소 초기화 완료
🆕 새로운 드라이버 초기화 중...
✅ 크롬 드라이버 실행 성공!
Windows
Windows
✅ 드라이버 초기화 완료!
📁 이미지 폴더 확인 완료 (기존 이미지 연속성 보장): c:\Users\redsk\OneDrive\デスクトップ\mikael_project\test_folder\myrealtripthumb_img
🌏 설정된 검색 도시: 다낭
  🌍 대륙: 아시아
  🏛️ 국가: 베트남
  ✈️ 공항 코드: DAD
⚙️ 크롤링 설정:
  📊 최대 상품 수: 6개
  ⏱️ 재시도 횟수: 3회
  🔄 대기 시간: 5-12초
  🖼️ 이미지 저장: ✅ 활성화
✅ 도시 유효성 검증: 지원하는 도시입니다. (DAD)
📁 데이터 저장 경로 생성 완료: data\아시아\베트남\다낭
📁 국가별 통합 경로 생성 완료: data\아시아\베트남

🔄 상태 관리 시스템 초기화 중...
✅ 상태 파일 로드: 0개 수집 완료
✅ completed_urls.log 정리 완료: 19개 고유 URL
✅ 완료된 URL 19개 로드
✅ 상태 관리 시스템 로드 완료
  📊 이전 수집 완료: 0개
  📝 완료된 URL: 19개
📄 기존 CSV 파일 없음 - 0부터 시작
🔢 번호 연속성 설정:
  📊 기존 마지막 번호: -1
  🆕 시작 번호: 0

🎉 그룹 6 완료: 시스템 초기화 성공!
📍 위치: 아시아 > 베트남 > 다낭
📁 이미지 저장 기본 경로: myrealtripthumb_img/아시아/베트남/다낭/
📁 데이터 저장 경로: data/아시아/베트남/다낭/
🔢 번호 시작점: 0
🚀 다음: 그룹 7을 실행하여 웹사이트 검색을 시작하세요!


In [7]:
# =============================================================================
# 🚀 그룹 7: 웹사이트 검색 및 페이지 이동
# - 마이리얼트립 사이트 접속, 도시 검색, 상품 목록 페이지 도달
# =============================================================================

print("🔍 그룹 7: 웹사이트 검색 및 페이지 이동 시작!")
print("=" * 60)

# 그룹 6에서 설정된 변수들 확인
try:
    print(f"📋 현재 설정 확인:")
    print(f"  🏙️ 검색 도시: {city_name}")
    print(f"  🌍 위치: {continent} > {country}")
    print(f"  ✈️ 공항 코드: {get_city_code(city_name)}")
    print(f"  🔢 시작 번호: {start_number}")
except NameError as e:
    print(f"❌ 그룹 6이 실행되지 않았습니다: {e}")
    print("💡 그룹 6을 먼저 실행하세요!")
    raise

print(f"\n🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)")
print(f"📍 현재 작업: {city_name} - 웹사이트 접속 중")
print("-" * 50)

try:
    # 1. 메인 페이지로 이동
    print("  📱 마이리얼트립 메인 페이지 이동 중...")
    retry_operation(
        lambda: go_to_main_page(driver), 
        "메인 페이지 이동"
    )
    print(f"  ✅ 마이리얼트립 페이지 열기 완료")
    
    # 진행률 업데이트
    print(f"\n🔍 진행률: [███████░░░░░░░░░░░░░░░░░░░░░░░] 25.0% (1/4)")
    print(f"📍 현재 작업: {city_name} - 검색창 입력 중")
    print("-" * 50)
    
    # 2. 검색창 찾기 및 입력 (🆕 동적 도시명 사용)
    retry_operation(
        lambda: find_and_fill_search(driver, city_name), 
        f"{city_name} 검색창 입력"
    )
    
    # 진행률 업데이트
    print(f"\n🔍 진행률: [██████████████░░░░░░░░░░░░░░░░] 50.0% (2/4)")
    print(f"📍 현재 작업: {city_name} - 검색 실행 중")
    print("-" * 50)

    # 3. 검색 버튼 클릭
    retry_operation(
        lambda: click_search_button(driver), 
        "검색 버튼 클릭"
    )
    
    # 진행률 업데이트
    print(f"\n🔍 진행률: [█████████████████████░░░░░░░░░] 75.0% (3/4)")
    print(f"📍 현재 작업: {city_name} - 페이지 최적화 중")
    print("-" * 50)

    # 4. 팝업 처리 (선택사항)
    try:
        handle_popup(driver)
    except Exception as e:
        print(f"  ℹ️ 팝업 처리 중 오류 (무시됨): {type(e).__name__}")

    # 5. 전체 상품 보기 클릭 (선택사항)
    try:
        click_view_all(driver)
    except Exception as e:
        print(f"  ℹ️ 전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")
    
    # 진행률 완료
    print(f"\n✅ 진행률: [██████████████████████████████] 100.0% (4/4)")
    print(f"📍 현재 작업: {city_name} - 완료")
    print("-" * 50)

    print(f"\n🎉 그룹 7 완료: {city_name} 검색 성공!")
    print(f"🎯 현재 상태: {city_name} 상품 목록 페이지에 도달했습니다")
    
    # 현재 페이지 URL 확인
    try:
        current_url = driver.current_url
        print(f"🔗 현재 페이지: {current_url[:80]}...")
        
        # URL에서 도시명 확인
        if city_name in current_url or get_city_code(city_name).lower() in current_url.lower():
            print("✅ URL 검증: 올바른 도시 페이지에 있습니다")
        else:
            print("⚠️ URL 검증: 예상과 다른 페이지일 수 있습니다")
            
    except Exception as e:
        print(f"⚠️ 현재 URL 확인 실패: {e}")
    
    # 페이지 로드 상태 확인
    try:
        page_state = driver.execute_script("return document.readyState;")
        if page_state == "complete":
            print("✅ 페이지 로드: 완전히 로드됨")
        else:
            print(f"⚠️ 페이지 로드: {page_state} 상태")
            print("  ⏰ 추가 로딩 대기 중...")
            time.sleep(3)
    except Exception as e:
        print(f"⚠️ 페이지 상태 확인 실패: {e}")
    
    print("🚀 다음: 그룹 8을 실행하여 URL 수집 및 분석을 시작하세요!")
    
except TimeoutException as e:
    print(f"\n⏰ {city_name}: 페이지 로딩 시간 초과")
    print(f"📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 7 실패: 시간 초과")
    print("💡 해결 방법:")
    print("   1. 인터넷 연결 상태 확인")
    print("   2. 마이리얼트립 사이트 접속 상태 확인")
    print("   3. 드라이버 재시작 후 재시도")
    
except NoSuchElementException as e:
    print(f"\n🔍 {city_name}: 필요한 웹 요소를 찾을 수 없음")
    print(f"📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 7 실패: 요소 없음")
    print("💡 해결 방법:")
    print("   1. 마이리얼트립 사이트 구조 변경 확인")
    print("   2. 검색하는 도시명이 정확한지 확인")
    print("   3. 브라우저를 수동으로 확인")
    
except Exception as e:
    print(f"\n❌ {city_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    print("❌ 그룹 7 실패: 예상치 못한 오류")
    print("💡 해결 방법:")
    print("   1. 그룹 6이 성공적으로 완료되었는지 확인")
    print(f"   2. CITIES_TO_SEARCH에 '{city_name}'이 올바르게 설정되었는지 확인")
    print("   3. 드라이버 상태 확인")
    print("   4. 브라우저 재시작 후 재시도")

print("\n" + "=" * 60)
print("✅ 그룹 7 웹사이트 검색 단계 완료!")
print("🔧 수행된 작업:")
print("   - 마이리얼트립 메인 페이지 접속")
print("   - 도시명 검색창 입력")
print("   - 검색 버튼 클릭")
print("   - 팝업 및 UI 최적화")
print("   - 상품 목록 페이지 도달")
print("🎯 다음 단계: 그룹 8에서 URL 수집 및 페이지네이션 분석")

🔍 그룹 7: 웹사이트 검색 및 페이지 이동 시작!
📋 현재 설정 확인:
  🏙️ 검색 도시: 다낭
  🌍 위치: 아시아 > 베트남
  ✈️ 공항 코드: DAD
  🔢 시작 번호: 0

🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 다낭 - 웹사이트 접속 중
--------------------------------------------------
  📱 마이리얼트립 메인 페이지 이동 중...
  ✅ 마이리얼트립 페이지 열기 완료

🔍 진행률: [███████░░░░░░░░░░░░░░░░░░░░░░░] 25.0% (1/4)
📍 현재 작업: 다낭 - 검색창 입력 중
--------------------------------------------------
  🔍 '다낭' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '다낭' 키워드 입력 완료

🔍 진행률: [██████████████░░░░░░░░░░░░░░░░] 50.0% (2/4)
📍 현재 작업: 다낭 - 검색 실행 중
--------------------------------------------------
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!

🔍 진행률: [█████████████████████░░░░░░░░░] 75.0% (3/4)
📍 현재 작업: 다낭 - 페이지 최적화 중
--------------------------------------------------
  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.
  📋 전체 상품 보기 버튼 찾는 중...
  ✅ 전체 상품 보기 클릭 성공!

✅ 진행률: [██████████████████████████████] 100.0% (4/4)
📍 현재 작업: 다낭 - 완료
--------------------------------------------------

🎉 그룹 7 완료: 다낭 검색 성공!
🎯 현재 상태: 다낭 상품 목록 페이지에 도달했습니다
🔗 현재 페이지

In [8]:
# =============================================================================
# 🚀 그룹 8: URL 수집 및 페이지네이션 분석
# - 세션 안전 URL 수집, 중복 방지, 페이지네이션 정보 분석
# =============================================================================

print("🔍 그룹 8: URL 수집 및 페이지네이션 분석 시작!")
print("=" * 60)

# 그룹 6, 7에서 설정된 변수들 확인
try:
    print(f"📋 현재 상태 확인:")
    print(f"  🏙️ 대상 도시: {city_name}")
    print(f"  📊 완료된 URL: {len(completed_urls)}개")
    print(f"  🔢 시작 번호: {start_number}")
    print(f"  📱 드라이버 상태: 활성")
except NameError as e:
    print(f"❌ 필수 변수가 설정되지 않았습니다: {e}")
    print("💡 그룹 6과 그룹 7을 먼저 실행하세요!")
    raise

print("\n🔍 === 1단계: 페이지네이션 정보 분석 ===")

try:
    # 페이지네이션 분석 실행
    print(f"🔍 {city_name} 페이지네이션 정보 분석 중...")
    pagination_info = analyze_pagination(driver)
    
    # 크롤링 계획 수립
    plan = generate_crawling_plan(pagination_info, city_name)
    can_proceed = report_reconnaissance_results(plan)
    
    # 다음 페이지 버튼 작동성 확인
    button_working = check_next_button(driver)
    
    if can_proceed and button_working:
        print("\n🚀 페이지네이션을 활용한 전체 크롤링이 가능합니다!")
        pagination_strategy = "다중 페이지"
    else:
        print("\n⚠️ 페이지네이션 제한으로 현재 페이지만 크롤링 가능합니다.")
        pagination_strategy = "단일 페이지"
        
except Exception as e:
    print(f"⚠️ 페이지네이션 분석 실패: {e}")
    print("💡 기본 전략으로 진행합니다.")
    pagination_strategy = "기본"
    pagination_info = {
        'total_products': 0,
        'total_pages': 1,
        'products_per_page': 24,
        'has_next_button': False,
        'is_pagination_available': False
    }

print(f"\n🔍 === 2단계: 세션 안전 URL 수집 ===")

try:
    # 세션 안전 URL 수집 실행
    print(f"📊 {city_name}에서 세션 안전 URL 수집을 시작합니다...")
    
    collected_urls = collect_urls_with_session_safety(driver, city_name, completed_urls)
    
    if collected_urls:
        print(f"\n🎉 새로운 URL 수집 성공!")
        print(f"📈 총 {len(collected_urls)}개의 새로운 URL을 찾았습니다!")
        
        # 설정된 개수와 비교
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        will_crawl = min(len(collected_urls), max_products)
        
        print(f"🔢 설정된 크롤링 개수: {max_products}개")
        print(f"🎯 실제 크롤링할 개수: {will_crawl}개")
        
        print(f"\n📋 수집된 새로운 URL 목록:")
        print("-" * 80)
        for i, url in enumerate(collected_urls[:max_products], 1):
            url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
            print(f"  {i:2d}. {url_type}: {url}")
        
        # URL 유효성 체크
        print(f"\n🔍 URL 유효성 체크:")
        products_count = sum(1 for url in collected_urls if '/products/' in url)
        offers_count = sum(1 for url in collected_urls if '/offers/' in url)
        
        print(f"  ✅ Products: {products_count}개")
        print(f"  ✅ Offers: {offers_count}개")
        print(f"  🎯 총 새로운 URL: {products_count + offers_count}개")
        
        # 전역 변수로 URL 저장 (그룹 9에서 사용)
        urls_to_crawl = collected_urls[:max_products]
        total_products_to_crawl = len(urls_to_crawl)
        
        print("\n🚀 세션 안전 URL 수집 성공! 다음 단계로 진행 가능합니다!")
        
    else:
        print("\n❌ 새로운 URL 수집 실패 또는 모든 URL이 이미 처리됨")
        print("💡 추가 분석:")
        
        # 기본 URL 수집으로 전체 상황 파악
        try:
            basic_urls = collect_all_24_urls(driver)
            print(f"   📄 페이지의 전체 URL: {len(basic_urls)}개")
            print(f"   📄 이미 완료된 URL: {len(completed_urls)}개")
            
            overlap_count = len([url for url in basic_urls if url in completed_urls])
            print(f"   🔄 중복 URL: {overlap_count}개")
            
            if overlap_count == len(basic_urls):
                print("   ✅ 모든 URL이 이미 처리되어 중복 방지가 정상 작동했습니다!")
                urls_to_crawl = []
                total_products_to_crawl = 0
            else:
                print("   ⚠️ 중복 방지 시스템에 문제가 있을 수 있습니다.")
                # 기본 URL이라도 새로운 것들 사용
                new_basic_urls = [url for url in basic_urls if url not in completed_urls]
                urls_to_crawl = new_basic_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
                total_products_to_crawl = len(urls_to_crawl)
                print(f"   🔄 기본 수집으로 {total_products_to_crawl}개 URL 확보")
                
        except Exception as debug_error:
            print(f"   ❌ 분석 실패: {debug_error}")
            urls_to_crawl = []
            total_products_to_crawl = 0
        
        if total_products_to_crawl == 0:
            print("🔄 새로운 페이지로 이동하거나 다른 도시를 시도해보세요!")

except Exception as e:
    print(f"❌ 세션 안전 URL 수집 중 오류: {type(e).__name__}: {e}")
    print("💡 가능한 해결책:")
    print("   1. 그룹 7이 성공적으로 완료되었는지 확인")
    print("   2. 브라우저가 올바른 상품 목록 페이지에 있는지 확인")
    print("   3. 인터넷 연결 상태 확인")
    
    # 오류 시 기본 URL 수집 시도
    try:
        print("\n🔄 기본 URL 수집 방식으로 재시도...")
        basic_urls = collect_all_24_urls(driver)
        new_urls = filter_new_urls(basic_urls, completed_urls)
        urls_to_crawl = new_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
        total_products_to_crawl = len(urls_to_crawl)
        print(f"✅ 기본 방식으로 {total_products_to_crawl}개 URL 확보")
    except Exception as fallback_error:
        print(f"❌ 기본 방식도 실패: {fallback_error}")
        urls_to_crawl = []
        total_products_to_crawl = 0

print(f"\n🔍 === 3단계: 수집 결과 요약 ===")

try:
    print(f"📊 수집 결과 요약:")
    print(f"  🏙️ 대상 도시: {city_name}")
    print(f"  📄 페이지네이션 전략: {pagination_strategy}")
    print(f"  🔢 수집된 신규 URL: {total_products_to_crawl}개")
    print(f"  📊 설정된 최대 크롤링: {CONFIG['MAX_PRODUCTS_PER_CITY']}개")
    print(f"  🔢 번호 시작점: {start_number}")
    
    if total_products_to_crawl > 0:
        print(f"  📈 예상 번호 범위: {start_number} ~ {start_number + total_products_to_crawl - 1}")
        print(f"  📁 이미지 저장 위치: myrealtripthumb_img/{continent}/{country}/{city_name}/")
        print(f"  📁 데이터 저장 위치: data/{continent}/{country}/{city_name}/")
        
        print(f"\n✅ 그룹 8 완료: URL 수집 및 분석 성공!")
        print("🚀 다음: 그룹 9를 실행하여 실제 크롤링을 시작하세요!")
    else:
        print(f"\n⚠️ 그룹 8 완료: 크롤링할 새로운 URL이 없음")
        print("💡 다음 단계:")
        print("   1. 다른 도시로 변경")
        print("   2. config/completed_urls.log 초기화")
        print("   3. 페이지네이션을 통한 다음 페이지 이동")
        
except NameError:
    print("❌ 필수 변수가 설정되지 않았습니다.")
    print("💡 URL 수집에 실패했을 가능성이 높습니다.")
    total_products_to_crawl = 0

print("\n" + "=" * 60)
print("✅ 그룹 8 URL 수집 및 페이지네이션 분석 완료!")
print("🔧 수행된 작업:")
print("   - 페이지네이션 정보 분석")
print("   - 크롤링 계획 수립")
print("   - 세션 안전 URL 수집")
print("   - URL 중복 방지 및 필터링")
print("   - 수집 결과 검증")
print("🛡️ 보안 기능:")
print("   - 세션 간 URL 중복 방지")
print("   - 완료된 작업 자동 제외")
print("   - 번호 연속성 보장")

if 'total_products_to_crawl' in locals() and total_products_to_crawl > 0:
    print(f"🎯 다음 단계: 그룹 9에서 {total_products_to_crawl}개 상품 크롤링 시작")

🔍 그룹 8: URL 수집 및 페이지네이션 분석 시작!
📋 현재 상태 확인:
  🏙️ 대상 도시: 다낭
  📊 완료된 URL: 19개
  🔢 시작 번호: 0
  📱 드라이버 상태: 활성

🔍 === 1단계: 페이지네이션 정보 분석 ===
🔍 다낭 페이지네이션 정보 분석 중...
  🔍 페이지네이션 정보 분석 중...
    ✅ 총 상품 수 발견: 819개

📋 크롤링 계획 수립 중...

🔍 === 정찰 완료 보고서 ===
📍 도시: 다낭
📊 발견된 총 상품 수: 819개
📄 총 페이지 수: 1페이지
📋 페이지당 상품 수: 24개
🔄 페이지네이션 가능: ❌ 아니오
⏱️ 예상 소요 시간: 3.0분
🎯 크롤링 전략: 단일 페이지
📦 실제 수집 예정: 6개
⚠️ 페이지네이션이 제한적입니다. 현재 페이지만 크롤링 가능합니다.
  🔍 다음 페이지 버튼 작동성 확인 중...
    ❌ 작동 가능한 다음 페이지 버튼을 찾을 수 없습니다.

⚠️ 페이지네이션 제한으로 현재 페이지만 크롤링 가능합니다.

🔍 === 2단계: 세션 안전 URL 수집 ===
📊 다낭에서 세션 안전 URL 수집을 시작합니다...
✅ URL 히스토리 업데이트: url_history/다낭.json

🎉 새로운 URL 수집 성공!
📈 총 24개의 새로운 URL을 찾았습니다!
🔢 설정된 크롤링 개수: 6개
🎯 실제 크롤링할 개수: 6개

📋 수집된 새로운 URL 목록:
--------------------------------------------------------------------------------
   1. 🏷️ Offer: https://www.myrealtrip.com/offers/63983
   2. 🛍️ Product: https://experiences.myrealtrip.com/products/3441466
   3. 🛍️ Product: https://experiences.myrealtrip.com/products/3795284
   4. 🏷️ Offer: https://www.

In [9]:
# =============================================================================
# 🚀 셀 11-A: 페이지네이션 핵심 시스템
# - 페이지네이션 상태 관리, URL 저장/복귀, 페이지 이동 함수들
# - 기존 그룹 1-8의 모든 연속성 보장 시스템 활용
# =============================================================================

import time
import random
import json
import os
from datetime import datetime

print("🔧 셀 11-A: 페이지네이션 핵심 시스템 로딩...")

# =============================================================================
# 📊 페이지네이션 상태 관리 시스템
# =============================================================================

def save_pagination_state(city_name, current_page, current_list_url, total_crawled, target_products):
    """
    페이지네이션 상태를 저장하여 세션 간 연속성 보장
    """
    try:
        os.makedirs("pagination_state", exist_ok=True)
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        pagination_state = {
            "city": city_name,
            "current_page": current_page,
            "current_list_url": current_list_url,
            "total_crawled": total_crawled,
            "target_products": target_products,
            "last_updated": datetime.now().isoformat(),
            "session_id": datetime.now().strftime('%Y%m%d_%H%M%S'),
            "status": "active"
        }
        
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(pagination_state, f, ensure_ascii=False, indent=2)
        
        print(f"      ✅ 페이지네이션 상태 저장: {current_page}페이지, {total_crawled}개 완료")
        return True
        
    except Exception as e:
        print(f"      ❌ 페이지네이션 상태 저장 실패: {e}")
        return False

def load_pagination_state(city_name):
    """
    이전 세션의 페이지네이션 상태 로드
    """
    try:
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        if not os.path.exists(state_file):
            print(f"      ℹ️ 페이지네이션 상태 파일 없음 - 새 세션 시작")
            return None
        
        with open(state_file, 'r', encoding='utf-8') as f:
            state = json.load(f)
        
        print(f"      ✅ 페이지네이션 상태 로드: {state.get('current_page', 1)}페이지부터 재개")
        print(f"      📊 이전 진행: {state.get('total_crawled', 0)}개 완료")
        
        return state
        
    except Exception as e:
        print(f"      ⚠️ 페이지네이션 상태 로드 실패: {e}")
        return None

def clear_pagination_state(city_name):
    """
    페이지네이션 상태 초기화 (크롤링 완료 시)
    """
    try:
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        if os.path.exists(state_file):
            # 완료 상태로 마킹
            with open(state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            
            state["status"] = "completed"
            state["completed_time"] = datetime.now().isoformat()
            
            with open(state_file, 'w', encoding='utf-8') as f:
                json.dump(state, f, ensure_ascii=False, indent=2)
            
            print(f"      ✅ 페이지네이션 상태 완료 처리")
        
        return True
        
    except Exception as e:
        print(f"      ⚠️ 페이지네이션 상태 정리 실패: {e}")
        return False

# =============================================================================
# 🔗 목록페이지 URL 저장 및 복귀 시스템
# =============================================================================

def save_list_page_url(driver, city_name, page_number):
    """
    현재 목록페이지 URL을 저장
    """
    try:
        current_url = driver.current_url
        
        # URL 검증
        if not is_valid_list_page_url(current_url, city_name):
            print(f"      ⚠️ 유효하지 않은 목록페이지 URL: {current_url}")
            return None
        
        print(f"      📝 {page_number}페이지 URL 저장: ...{current_url[-50:]}")
        return current_url
        
    except Exception as e:
        print(f"      ❌ 목록페이지 URL 저장 실패: {e}")
        return None

def return_to_list_page(driver, saved_url, city_name, max_attempts=3):
    """
    저장된 목록페이지 URL로 안전하게 복귀
    """
    print(f"      🔙 목록페이지로 복귀 중...")
    
    for attempt in range(max_attempts):
        try:
            current_url = driver.current_url
            
            # 이미 목록페이지에 있는지 확인
            if is_valid_list_page_url(current_url, city_name):
                # 상품 링크 개수로 확인
                product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
                if len(product_links) >= 10:
                    print(f"      ✅ 이미 올바른 목록페이지에 있음 ({len(product_links)}개 상품)")
                    return True, current_url
            
            print(f"      🔄 목록페이지 복귀 시도 {attempt + 1}/{max_attempts}")
            
            if saved_url:
                # 저장된 URL로 직접 이동
                driver.get(saved_url)
                print(f"      📍 저장된 URL로 이동: ...{saved_url[-50:]}")
            else:
                # 뒤로가기 시도
                driver.back()
                print(f"      ⬅️ 뒤로가기 시도")
            
            time.sleep(random.uniform(3, 5))
            
            # 복귀 확인
            new_url = driver.current_url
            product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
            
            if len(product_links) >= 10:
                print(f"      ✅ 목록페이지 복귀 성공! ({len(product_links)}개 상품 확인)")
                return True, new_url
            else:
                print(f"      ⚠️ 복귀했지만 상품 수 부족: {len(product_links)}개")
                
        except Exception as e:
            print(f"      ❌ 복귀 시도 {attempt + 1} 실패: {e}")
            
        if attempt < max_attempts - 1:
            print(f"      ⏰ 2초 후 재시도...")
            time.sleep(2)
    
    print(f"      🚨 목록페이지 복귀 최종 실패")
    return False, None

def is_valid_list_page_url(url, city_name):
    """
    유효한 목록페이지 URL인지 확인
    """
    if not url:
        return False
    
    # 상품 상세페이지가 아닌지 확인
    if "/products/" in url and url.count("/") > 5:
        return False
    if "/offers/" in url and url.count("/") > 5:
        return False
    
    # 기본 목록페이지 패턴 확인
    valid_patterns = [
        "/experiences",
        "/offers",
        "/search"
    ]
    
    return any(pattern in url for pattern in valid_patterns)

# =============================================================================
# 🔄 페이지 이동 및 복구 시스템
# =============================================================================

def click_next_page_enhanced(driver, current_page=None):
    """
    다음 페이지로 안전하게 이동 (강화된 버전)
    """
    print(f"    🔍 다음 페이지 버튼 찾는 중...")
    
    try:
        # 현재 상태 저장
        current_url = driver.current_url
        current_products = len(collect_all_24_urls(driver))
        
        # 다음 페이지 버튼 셀렉터들 (마닐라 최적화 + 범용)
        next_button_selectors = [
            # 마닐라 특화 셀렉터
            (By.XPATH, "/html/body/div[4]/div[2]/div/div/div[2]/main/div/div[4]/div/li[7]/button"),
            (By.CSS_SELECTOR, "button.css-13fjuep"),
            (By.XPATH, "//img[@src='https://dffoxz5he03rp.cloudfront.net/icons/ic_arrowright_sm_blue_500.svg']/ancestor::button"),
            
            # 범용 셀렉터들
            (By.XPATH, "//button[.//img[contains(@src, 'arrow') and contains(@src, 'right')]]"),
            (By.XPATH, "//button[contains(@aria-label, '다음') and not(@disabled)]"),
            (By.XPATH, "//button[contains(text(), '다음') and not(@disabled)]"),
            (By.CSS_SELECTOR, "button[aria-label*='next']:not([disabled])"),
            (By.XPATH, "//button[contains(@class, 'pagination')]//img[contains(@src, 'arrow')]"),
            (By.XPATH, "//li[last()]//button[not(@disabled)]"),  # 마지막 li의 버튼
        ]
        
        next_button = None
        used_selector = None
        
        # 버튼 찾기
        for selector_type, selector in next_button_selectors:
            try:
                buttons = driver.find_elements(selector_type, selector)
                for button in buttons:
                    if button.is_displayed() and button.is_enabled():
                        # disabled 속성 추가 체크
                        disabled = button.get_attribute("disabled")
                        aria_disabled = button.get_attribute("aria-disabled")
                        
                        if not disabled and aria_disabled != "true":
                            next_button = button
                            used_selector = selector
                            break
                
                if next_button:
                    break
                    
            except Exception:
                continue
        
        if not next_button:
            print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다")
            return False, "다음 페이지 버튼 없음", current_url
        
        print(f"    ✅ 다음 페이지 버튼 발견!")
        
        # 안전한 클릭 실행
        for click_attempt in range(3):
            try:
                print(f"    🖱️ 다음 페이지 클릭 시도 {click_attempt + 1}/3...")
                
                # 스크롤 후 클릭
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                
                # JavaScript 클릭
                driver.execute_script("arguments[0].click();", next_button)
                
                # 페이지 로딩 대기
                print(f"    ⏰ 페이지 로딩 대기 중...")
                time.sleep(random.uniform(4, 7))
                
                # 변화 검증
                new_url = driver.current_url
                new_products = len(collect_all_24_urls(driver))
                
                # 성공 조건 확인
                if new_url != current_url:
                    print(f"    ✅ URL 변화 감지: 페이지 이동 성공!")
                    print(f"    📍 새 URL: ...{new_url[-60:]}")
                    return True, "페이지 이동 성공", new_url
                    
                elif new_products != current_products and new_products > 0:
                    print(f"    ✅ 상품 수 변화 감지: {current_products} → {new_products}")
                    return True, "상품 변화로 이동 확인", new_url
                    
                else:
                    print(f"    ⚠️ 클릭 {click_attempt+1}: 페이지 변화 미감지")
                    if click_attempt < 2:
                        time.sleep(2)
                        continue
                
            except Exception as e:
                print(f"    ❌ 클릭 시도 {click_attempt+1} 실패: {e}")
                if click_attempt < 2:
                    time.sleep(2)
                    continue
        
        # 모든 시도 실패
        print(f"    🏁 마지막 페이지이거나 페이지 이동 실패")
        return False, "페이지 이동 실패 (3회 시도 후)", current_url
        
    except Exception as e:
        print(f"    ❌ 다음 페이지 이동 중 오류: {e}")
        return False, f"오류: {type(e).__name__}", driver.current_url

def attempt_page_recovery(driver, city_name, target_page):
    """
    페이지 오류 복구 시도 (도시별 URL 지원)
    """
    print(f"    🔧 페이지 복구 시도: {target_page}페이지")
    
    try:
        # 도시별 기본 목록페이지 URL 매핑
        city_base_urls = {
            "마닐라": "https://www.myrealtrip.com/offers?t=llp&qct=Manila&qcr=Philippines",
            "바르셀로나": "https://www.myrealtrip.com/experiences/?destination=%EB%B0%94%EB%A5%B4%EC%85%80%EB%A1%9C%EB%82%98",
            "방콕": "https://www.myrealtrip.com/experiences/?destination=%EB%B0%A9%EC%BD%95",
            "파리": "https://www.myrealtrip.com/experiences/?destination=%ED%8C%8C%EB%A6%AC",
            # 필요시 추가...
        }
        
        base_url = city_base_urls.get(city_name, "https://www.myrealtrip.com/experiences/")
        
        # 페이지 번호 추가
        if target_page > 1:
            separator = "&" if "?" in base_url else "?"
            recovery_url = f"{base_url}{separator}page={target_page}"
        else:
            recovery_url = base_url
        
        print(f"    📍 복구 URL로 이동: ...{recovery_url[-60:]}")
        driver.get(recovery_url)
        time.sleep(random.uniform(5, 8))
        
        # 복구 확인
        product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
        
        if len(product_links) >= 10:
            print(f"    ✅ 페이지 복구 성공! ({len(product_links)}개 상품 확인)")
            return True, recovery_url
        else:
            print(f"    ❌ 페이지 복구 실패: 상품 {len(product_links)}개만 발견")
            return False, None
            
    except Exception as e:
        print(f"    ❌ 페이지 복구 오류: {e}")
        return False, None

# =============================================================================
# 🧰 유틸리티 함수들
# =============================================================================

def validate_pagination_environment():
    """
    페이지네이션 실행 환경 검증
    """
    print("🔍 페이지네이션 환경 검증 중...")
    
    # 필수 변수들 확인
    required_vars = ['driver', 'city_name', 'start_number', 'completed_urls', 'continent', 'country']
    missing_vars = []
    
    for var_name in required_vars:
        if var_name not in globals():
            missing_vars.append(var_name)
    
    if missing_vars:
        print(f"❌ 필수 변수 누락: {', '.join(missing_vars)}")
        print("💡 그룹 6-8을 먼저 실행하세요!")
        return False, missing_vars
    
    # 드라이버 상태 확인
    try:
        current_url = driver.current_url
        print(f"✅ 드라이버 상태: 정상 ({current_url[:50]}...)")
    except:
        print(f"❌ 드라이버 상태: 비정상")
        return False, ["driver_inactive"]
    
    # 현재 페이지가 목록페이지인지 확인
    try:
        product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
        if len(product_links) >= 5:
            print(f"✅ 목록페이지 확인: {len(product_links)}개 상품 링크")
        else:
            print(f"⚠️ 목록페이지 의심: {len(product_links)}개 상품 링크만 발견")
    except:
        print(f"⚠️ 페이지 상태 확인 실패")
    
    print(f"✅ 페이지네이션 환경 검증 완료!")
    return True, []

def get_pagination_progress_info(total_crawled, target_products, current_page):
    """
    페이지네이션 진행 상황 정보 생성
    """
    progress_percentage = (total_crawled / target_products * 100) if target_products > 0 else 0
    remaining = max(0, target_products - total_crawled)
    
    return {
        'total_crawled': total_crawled,
        'target_products': target_products,
        'remaining': remaining,
        'progress_percentage': progress_percentage,
        'current_page': current_page,
        'estimated_pages': (target_products // 24) + (1 if target_products % 24 > 0 else 0)
    }

def print_pagination_progress(progress_info):
    """
    페이지네이션 진행률 시각적 표시
    """
    percentage = progress_info['progress_percentage']
    bar_length = 30
    filled_length = int(bar_length * progress_info['total_crawled'] // progress_info['target_products'])
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    print(f"\n📊 페이지네이션 진행률: [{bar}] {percentage:.1f}%")
    print(f"   🎯 진행: {progress_info['total_crawled']}/{progress_info['target_products']}개")
    print(f"   📄 페이지: {progress_info['current_page']}페이지")
    print(f"   ⏱️ 남은 상품: {progress_info['remaining']}개")
    print("-" * 60)

# =============================================================================
# 🎯 시스템 초기화 완료
# =============================================================================

print("✅ 셀 11-A: 페이지네이션 핵심 시스템 로드 완료!")
print()
print("🔧 로드된 핵심 함수들:")
print("   📊 save_pagination_state() - 페이지네이션 상태 저장")
print("   📊 load_pagination_state() - 페이지네이션 상태 로드")
print("   🔗 save_list_page_url() - 목록페이지 URL 저장")
print("   🔙 return_to_list_page() - 목록페이지 안전 복귀")
print("   🔄 click_next_page_enhanced() - 강화된 다음페이지 이동")
print("   🔧 attempt_page_recovery() - 페이지 오류 복구")
print("   🧰 validate_pagination_environment() - 환경 검증")
print()
print("🚀 다음: 셀 11-B를 실행하여 완전한 페이지네이션 크롤링 시작!")

🔧 셀 11-A: 페이지네이션 핵심 시스템 로딩...
✅ 셀 11-A: 페이지네이션 핵심 시스템 로드 완료!

🔧 로드된 핵심 함수들:
   📊 save_pagination_state() - 페이지네이션 상태 저장
   📊 load_pagination_state() - 페이지네이션 상태 로드
   🔗 save_list_page_url() - 목록페이지 URL 저장
   🔙 return_to_list_page() - 목록페이지 안전 복귀
   🔄 click_next_page_enhanced() - 강화된 다음페이지 이동
   🔧 attempt_page_recovery() - 페이지 오류 복구
   🧰 validate_pagination_environment() - 환경 검증

🚀 다음: 셀 11-B를 실행하여 완전한 페이지네이션 크롤링 시작!


In [10]:
# =============================================================================
# 🚀 셀 11-B: 완전한 페이지네이션 크롤링 실행
# - 메인 페이지네이션 크롤링 루프, 단일 상품 크롤링, 테스트 함수들
# - 셀 11-A의 핵심 시스템 + 기존 그룹 1-8의 모든 함수들 활용
# =============================================================================

import time
import random
from datetime import datetime

print("🔧 셀 11-B: 완전한 페이지네이션 크롤링 실행 로딩...")

# =============================================================================
# 🎯 메인 페이지네이션 크롤링 시스템
# =============================================================================

def crawl_with_full_pagination(city_name, target_products=100, resume_session=True):
    """
    🎯 완전한 페이지네이션 크롤링 메인 함수
    
    Args:
        city_name: 크롤링할 도시명
        target_products: 목표 상품 수
        resume_session: 이전 세션 이어서 실행 여부
    """
    print(f"🚀 완전한 페이지네이션 크롤링 시작!")
    print(f"🎯 목표: {city_name} {target_products}개 상품")
    print("=" * 80)
    
    # 1. 환경 검증
    env_valid, missing = validate_pagination_environment()
    if not env_valid:
        print(f"❌ 환경 검증 실패: {missing}")
        return False
    
    # 2. 전역 변수 활용 (그룹 6-8에서 설정됨)
    global start_number, completed_urls, continent, country, driver
    
    # 3. 세션 상태 관리
    current_page = 1
    total_crawled = 0
    current_product_number = start_number
    all_results = []
    
    # 이전 세션 복원 시도
    if resume_session:
        prev_state = load_pagination_state(city_name)
        if prev_state and prev_state.get("status") != "completed":
            current_page = prev_state.get("current_page", 1)
            print(f"🔄 이전 세션 복원: {current_page}페이지부터 재개")
            
            # 해당 페이지로 복구 시도
            if current_page > 1:
                recovery_success, recovery_url = attempt_page_recovery(driver, city_name, current_page)
                if not recovery_success:
                    print(f"⚠️ 페이지 복구 실패 - 1페이지부터 재시작")
                    current_page = 1
    
    print(f"📊 크롤링 설정:")
    print(f"   🏙️ 도시: {city_name}")
    print(f"   🎯 목표: {target_products}개")
    print(f"   🔢 시작 번호: {current_product_number}")
    print(f"   📄 시작 페이지: {current_page}")
    print(f"   📍 위치: {continent}/{country}")
    
    # 4. 메인 페이지네이션 루프
    while total_crawled < target_products:
        print(f"\n📄 === {current_page}페이지 처리 시작 ===")
        
        try:
            # 현재 목록페이지 URL 저장
            current_list_url = save_list_page_url(driver, city_name, current_page)
            if not current_list_url:
                print(f"❌ 목록페이지 URL 저장 실패")
                break
            
            # 현재 페이지의 URL 수집
            print(f"📊 {current_page}페이지 URL 수집 중...")
            page_urls = collect_all_24_urls(driver)
            
            if not page_urls:
                print(f"⚠️ {current_page}페이지: URL 없음")
                break
            
            print(f"✅ {current_page}페이지에서 {len(page_urls)}개 URL 발견")
            
            # 남은 목표 수량만큼만 처리
            remaining_target = min(2, target_products - total_crawled)  # 페이지당 2개 제한/테스트 후 롤백
            urls_to_process = page_urls[:remaining_target]
            
            print(f"🎯 {current_page}페이지에서 {len(urls_to_process)}개 상품 크롤링 시작...")
            
            # 5. 페이지 내 상품들 순차 크롤링
            page_results = []
            
            for url_index, product_url in enumerate(urls_to_process):
                print(f"    📦 상품 {current_product_number} 처리 중... ({url_index+1}/{len(urls_to_process)})")
                
                # 단일 상품 크롤링 실행
                result = crawl_single_product_optimized(
                    driver, product_url, current_product_number, 
                    city_name, continent, country, current_page
                )
                
                if result:
                    page_results.append(result)
                    total_crawled += 1
                    current_product_number += 1
                    
                    print(f"    ✅ 상품 {current_product_number-1} 완료: {result.get('상품명', '')[:30]}...")
                    
                    # 상태 저장 (각 상품 완료 시)
                    save_crawler_state(crawler_state, product_url)
                    
                    # 목표 달성 확인
                    if total_crawled >= target_products:
                        print(f"    🎉 목표 {target_products}개 달성! 크롤링 완료")
                        break
                else:
                    print(f"    ❌ 상품 {current_product_number} 처리 실패")
                    current_product_number += 1
                    continue
            
            # 6. 페이지별 배치 저장
            if page_results:
                batch_result = save_batch_data(page_results, city_name)
                if batch_result:
                    print(f"✅ {current_page}페이지 배치 저장 완료: {len(page_results)}개")
                all_results.extend(page_results)
            
            # 7. 페이지네이션 상태 저장
            save_pagination_state(city_name, current_page, current_list_url, total_crawled, target_products)
            
            # 8. 진행률 표시
            progress_info = get_pagination_progress_info(total_crawled, target_products, current_page)
            print_pagination_progress(progress_info)
            
            print(f"📄 {current_page}페이지 완료: {len(page_results)}개 수집 (전체: {total_crawled}/{target_products})")
            
            # 9. 목표 달성 확인
            if total_crawled >= target_products:
                print(f"🎉 목표 {target_products}개 달성! 페이지네이션 크롤링 완료")
                break
            
            # 10. 다음 페이지로 이동
            print(f"🔄 다음 페이지 이동 준비...")
            
            # 목록페이지로 복귀
            return_success, new_list_url = return_to_list_page(driver, current_list_url, city_name)
            
            if not return_success:
                print(f"❌ 목록페이지 복귀 실패 - 복구 시도")
                recovery_success, recovery_url = attempt_page_recovery(driver, city_name, current_page)
                if not recovery_success:
                    print(f"🚨 복구 실패: 크롤링 중단")
                    break
                new_list_url = recovery_url
            
            # 다음 페이지 이동
            next_success, next_message, next_url = click_next_page_enhanced(driver, current_page)
            
            if next_success:
                current_page += 1
                print(f"✅ {current_page}페이지로 이동 완료")
                time.sleep(random.uniform(2, 4))
            else:
                if "마지막 페이지" in next_message or "버튼 없음" in next_message:
                    print(f"🏁 마지막 페이지 도달: {next_message}")
                    break
                else:
                    print(f"❌ 페이지 이동 실패: {next_message}")
                    # 복구 시도
                    recovery_success, recovery_url = attempt_page_recovery(driver, city_name, current_page + 1)
                    if recovery_success:
                        current_page += 1
                        print(f"✅ 복구로 {current_page}페이지 이동")
                    else:
                        print(f"🚨 복구 실패: 크롤링 중단")
                        break
        
        except Exception as e:
            print(f"❌ {current_page}페이지 처리 중 오류: {e}")
            
            # 페이지 오류 복구 시도
            recovery_success, recovery_url = attempt_page_recovery(driver, city_name, current_page)
            if recovery_success:
                print(f"✅ 페이지 오류 복구 성공")
                continue
            else:
                print(f"🚨 페이지 오류 복구 실패: 크롤링 중단")
                break
    
    # 11. 최종 완료 처리
    clear_pagination_state(city_name)
    
    print(f"\n🎉 완전한 페이지네이션 크롤링 완료!")
    print(f"📊 최종 결과:")
    print(f"   🎯 목표/실제: {target_products}/{total_crawled}개")
    print(f"   📄 처리 페이지: {current_page}페이지")
    print(f"   🔢 번호 범위: {start_number} ~ {current_product_number - 1}")
    print(f"   📁 저장 위치: data/{continent}/{country}/{city_name}/")
    print(f"   📷 이미지 위치: myrealtripthumb_img/{continent}/{country}/{city_name}/")
    
    return {
        'success': True,
        'total_crawled': total_crawled,
        'pages_processed': current_page,
        'final_product_number': current_product_number - 1,
        'results': all_results
    }

# =============================================================================
# 🔧 단일 상품 크롤링 함수 (기존 그룹 1-2 함수들 활용)
# =============================================================================

def crawl_single_product_optimized(driver, product_url, product_number, city_name, continent, country, page_num):
    """
    단일 상품 크롤링 최적화 버전 (기존 함수들 재활용)
    """
    try:
        # 상품 페이지 이동
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        
        # URL 타입 판별
        url_type = "Product" if "/products/" in product_url else "Offer"
        
        # 정보 수집 (기존 그룹 1-2 함수들 활용)
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        rating_raw = get_rating(driver)
        rating_clean = clean_rating(rating_raw)
        review_count = get_review_count(driver)
        language = get_language(driver)
        
        # 이미지 다운로드 (기존 함수 활용)
        if CONFIG["SAVE_IMAGES"]:
            img_info = download_image(driver, product_name, city_name, product_number)
        else:
            img_info = {
                'filename': '', 'relative_path': '', 'path': '', 
                'status': 'skipped', 'size': 0
            }
        
        # 결과 반환 (기존 구조 유지)
        return {
            '번호': product_number,
            '페이지': page_num,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': get_city_code(city_name),
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language,
            '이미지_파일명': img_info.get('filename', ''),
            '이미지_상대경로': img_info.get('relative_path', ''),
            '이미지_전체경로': img_info.get('path', ''),
            '이미지_상태': img_info.get('status', ''),
            '이미지_크기': img_info.get('size', 0),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        
    except Exception as e:
        print(f"      ❌ 상품 크롤링 실패: {e}")
        return None

# =============================================================================
# 🧪 테스트 및 실행 함수들
# =============================================================================

def test_pagination_4_products():
    """
    4개 상품 페이지네이션 테스트 (사용자 요청 시나리오)
    """
    print("🧪 4개 상품 페이지네이션 테스트 시작!")
    print("📋 예상 시나리오: 1페이지 2개 → 2페이지 2개")
    print("=" * 60)
    
    # 기본 검증
    if 'city_name' not in globals():
        print("❌ city_name이 설정되지 않았습니다. 그룹 6-8을 먼저 실행하세요!")
        return False
    
    # 4개 상품 크롤링 실행
    result = crawl_with_full_pagination(city_name, target_products=4, resume_session=False)
    
    if result and result.get('success'):
        print(f"\n🎉 4개 상품 테스트 성공!")
        print(f"📊 결과:")
        print(f"   ✅ 수집: {result['total_crawled']}개")
        print(f"   📄 페이지: {result['pages_processed']}페이지")
        print(f"   🔢 번호: {start_number} ~ {result['final_product_number']}")
        print(f"   📁 저장: data/{continent}/{country}/{city_name}/")
        
        # 연속성 검증
        print(f"\n🔍 연속성 검증:")
        expected_numbers = list(range(start_number, start_number + 4))
        print(f"   📊 예상 번호: {expected_numbers}")
        print(f"   ✅ 이미지 파일: {get_city_code(city_name)}_0000.jpg ~ {get_city_code(city_name)}_0003.jpg")
        
        return result
    else:
        print(f"❌ 4개 상품 테스트 실패")
        return False

def test_pagination_100_products():
    """
    100개 상품 페이지네이션 테스트 (전체 시나리오)
    """
    print("🧪 100개 상품 페이지네이션 테스트 시작!")
    print("📋 예상: 약 5페이지 처리 (24+24+24+24+4)")
    print("=" * 60)
    
    if 'city_name' not in globals():
        print("❌ city_name이 설정되지 않았습니다. 그룹 6-8을 먼저 실행하세요!")
        return False
    
    # 100개 상품 크롤링 실행
    result = crawl_with_full_pagination(city_name, target_products=100, resume_session=True)
    
    if result and result.get('success'):
        print(f"\n🎉 100개 상품 테스트 성공!")
        print(f"📊 결과: {result['total_crawled']}개 수집, {result['pages_processed']}페이지")
        return result
    else:
        print(f"❌ 100개 상품 테스트 실패")
        return False

def run_custom_pagination_test(target_products=None):
    """
    사용자 지정 개수 페이지네이션 테스트
    """
    if target_products is None:
        target_products = CONFIG.get('MAX_PRODUCTS_PER_CITY', 50)
    
    print(f"🧪 사용자 지정 페이지네이션 테스트: {target_products}개")
    print("=" * 60)
    
    if 'city_name' not in globals():
        print("❌ 환경이 준비되지 않았습니다. 그룹 6-8을 먼저 실행하세요!")
        return False
    
    result = crawl_with_full_pagination(city_name, target_products=target_products, resume_session=True)
    
    if result and result.get('success'):
        print(f"\n🎉 {target_products}개 상품 테스트 성공!")
        print(f"📊 결과: {result['total_crawled']}개 수집, {result['pages_processed']}페이지")
        return result
    else:
        print(f"❌ {target_products}개 상품 테스트 실패")
        return False

def quick_pagination_status_check():
    """
    현재 페이지네이션 상태 빠른 확인
    """
    print("🔍 페이지네이션 상태 빠른 확인")
    print("-" * 40)
    
    # 환경 체크
    env_valid, missing = validate_pagination_environment()
    if env_valid:
        print("✅ 환경: 정상")
    else:
        print(f"❌ 환경: 문제 ({missing})")
        return
    
    # 현재 페이지 정보
    try:
        current_url = driver.current_url
        product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
        print(f"📍 현재 URL: ...{current_url[-50:]}")
        print(f"📊 상품 링크: {len(product_links)}개")
        
        # 페이지네이션 버튼 확인
        next_buttons = driver.find_elements(By.XPATH, "//button[contains(@aria-label, '다음') or .//img[contains(@src, 'arrow')]]")
        print(f"🔄 다음페이지 버튼: {len(next_buttons)}개")
        
    except Exception as e:
        print(f"❌ 상태 확인 실패: {e}")
    
    # 저장된 상태 확인
    if 'city_name' in globals():
        saved_state = load_pagination_state(city_name)
        if saved_state:
            print(f"💾 저장된 상태: {saved_state.get('current_page', 'N/A')}페이지, {saved_state.get('total_crawled', 0)}개 완료")
        else:
            print(f"💾 저장된 상태: 없음")

# =============================================================================
# 🎯 시스템 로드 완료
# =============================================================================

print("✅ 셀 11-B: 완전한 페이지네이션 크롤링 실행 로드 완료!")
print()
print("🚀 사용 가능한 함수들:")
print("   🎯 crawl_with_full_pagination() - 메인 페이지네이션 크롤링")
print("   🧪 test_pagination_4_products() - 4개 상품 테스트")
print("   🧪 test_pagination_100_products() - 100개 상품 테스트")
print("   🔧 run_custom_pagination_test() - 사용자 지정 개수 테스트")
print("   🔍 quick_pagination_status_check() - 상태 빠른 확인")
print()
print("📋 권장 실행 순서:")
print("   1️⃣ 그룹 1-8 실행 (환경 준비)")
print("   2️⃣ quick_pagination_status_check() (상태 확인)")
print("   3️⃣ test_pagination_4_products() (테스트)")
print("   4️⃣ 성공 시 더 큰 규모 실행")
print()
print("🎉 완전한 페이지네이션 시스템 준비 완료!")
print("=" * 80)

🔧 셀 11-B: 완전한 페이지네이션 크롤링 실행 로딩...
✅ 셀 11-B: 완전한 페이지네이션 크롤링 실행 로드 완료!

🚀 사용 가능한 함수들:
   🎯 crawl_with_full_pagination() - 메인 페이지네이션 크롤링
   🧪 test_pagination_4_products() - 4개 상품 테스트
   🧪 test_pagination_100_products() - 100개 상품 테스트
   🔧 run_custom_pagination_test() - 사용자 지정 개수 테스트
   🔍 quick_pagination_status_check() - 상태 빠른 확인

📋 권장 실행 순서:
   1️⃣ 그룹 1-8 실행 (환경 준비)
   2️⃣ quick_pagination_status_check() (상태 확인)
   3️⃣ test_pagination_4_products() (테스트)
   4️⃣ 성공 시 더 큰 규모 실행

🎉 완전한 페이지네이션 시스템 준비 완료!


In [12]:
quick_pagination_status_check()

🔍 페이지네이션 상태 빠른 확인
----------------------------------------
🔍 페이지네이션 환경 검증 중...
✅ 드라이버 상태: 정상 (https://www.myrealtrip.com/offers?t=llp&qct=Da%20N...)
✅ 목록페이지 확인: 24개 상품 링크
✅ 페이지네이션 환경 검증 완료!
✅ 환경: 정상
📍 현재 URL: ...trip.com/offers?t=llp&qct=Da%20Nang&qcr=Viet%20Nam
📊 상품 링크: 24개
🔄 다음페이지 버튼: 20개
      ℹ️ 페이지네이션 상태 파일 없음 - 새 세션 시작
💾 저장된 상태: 없음


In [13]:
test_pagination_4_products()

🧪 4개 상품 페이지네이션 테스트 시작!
📋 예상 시나리오: 1페이지 2개 → 2페이지 2개
🚀 완전한 페이지네이션 크롤링 시작!
🎯 목표: 다낭 4개 상품
🔍 페이지네이션 환경 검증 중...
✅ 드라이버 상태: 정상 (https://www.myrealtrip.com/offers?t=llp&qct=Da%20N...)
✅ 목록페이지 확인: 24개 상품 링크
✅ 페이지네이션 환경 검증 완료!
📊 크롤링 설정:
   🏙️ 도시: 다낭
   🎯 목표: 4개
   🔢 시작 번호: 0
   📄 시작 페이지: 1
   📍 위치: 아시아/베트남

📄 === 1페이지 처리 시작 ===
      📝 1페이지 URL 저장: ...trip.com/offers?t=llp&qct=Da%20Nang&qcr=Viet%20Nam
📊 1페이지 URL 수집 중...
📊 현재 페이지의 모든 URL 수집 중...
✅ 총 24개 URL 수집 완료!
   🛍️ Products: 15개
   🏷️ Offers: 9개
✅ 1페이지에서 24개 URL 발견
🎯 1페이지에서 2개 상품 크롤링 시작...
    📦 상품 0 처리 중... (1/2)
  📊 Offer 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '31,500원'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 2,304개 전체 보기
  🌐 언어 정보 찾는 중...
    ✅ 언어 정보 발견: 한국어
  🖼️ 대표 상품 이미지 다운로드 중...
    ✅ 이미지 URL 발견: https://d2ur7st6jjikze.cloudfront.net/offer_descri...
    📁 계층 폴더 생성: myrealtripthumb_img\아시아\베트남\다낭
    ✅ 계층 구조 이미지 저장 완료! (79,869 bytes)
    📍 저장 위치: 아시아\베트남\다낭\DAD_0000.jpg
    ✅ 상품 0 완료: 다낭 바나힐 입장권 티켓 QR코드 바로 입장 (왕복 케...
    📦 상품

{'success': True,
 'total_crawled': 4,
 'pages_processed': 2,
 'final_product_number': 3,
 'results': [{'번호': 0,
   '페이지': 1,
   '대륙': '아시아',
   '국가': '베트남',
   '도시': '다낭',
   '공항코드': 'DAD',
   '상품타입': 'Offer',
   '상품명': '다낭 바나힐 입장권 티켓 QR코드 바로 입장 (왕복 케이블카 포함)',
   '가격_원본': '31,500원',
   '가격_정제': '31,500원',
   '평점_원본': '4.8',
   '평점_정제': 4.8,
   '리뷰수': '후기 2,304개 전체 보기',
   '언어': '한국어',
   '이미지_파일명': 'DAD_0000.jpg',
   '이미지_상대경로': '아시아\\베트남\\다낭\\DAD_0000.jpg',
   '이미지_전체경로': 'myrealtripthumb_img\\아시아\\베트남\\다낭\\DAD_0000.jpg',
   '이미지_상태': '다운로드 완료',
   '이미지_크기': 79869,
   'URL': 'https://www.myrealtrip.com/offers/63983',
   '수집_시간': '2025-07-26 23:49:21',
   '상태': '완전수집'},
  {'번호': 1,
   '페이지': 1,
   '대륙': '아시아',
   '국가': '베트남',
   '도시': '다낭',
   '공항코드': 'DAD',
   '상품타입': 'Product',
   '상품명': '[10%특가] 바나힐 체크아웃 단독투어(12시간차량,케이블카,골든브릿지,공항/호텔드랍가능)',
   '가격_원본': '16,900원~',
   '가격_정제': '16,900원',
   '평점_원본': '4.6 ·',
   '평점_정제': 4.6,
   '리뷰수': '후기 288개',
   '언어': '한국어',
   '이미지_파일명': 'DAD_000